In [ ]:

#SECTION A: Greening and Browning Mapping

#  Always import cartopy before geemap to avoid widget conflicts
import ee
import geemap, ipyleaflet
import matplotlib.pyplot as plt
from geemap import cartoee

print("All required libraries (cartopy, geemap, ee, matplotlib) loaded successfully!")

ee.Authenticate()

ee.Initialize(project='danielsiglab')





In [ ]:

import ee

# 1. Load All Tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50") (splitting our study area in tiles)

# 2. Load Full Ecozone Layer

full_ecozone_fc = ee.FeatureCollection("projects/danielsiglab/assets/studyarea50")

# 2. Rasterize Ecozone Field
#ecozone_raster = full_ecozone_fc.reduceToImage(
  #  properties=['ECOZONE_ID'],
   # reducer=ee.Reducer.first()
#).rename('Ecozone

ecozone_raster = full_ecozone_fc.reduceToImage(
  properties=['ID'],
   reducer=ee.Reducer.first()
).rename('Ecozone')


slope_green_image = ecozone_raster.remap(
    [3,4,5,6,9,12,14,15,16],
    [0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015 ]
).rename('SlopeGreen')

slope_brown_image = ecozone_raster.remap(
    [3,4,5,6,9,12,14,15,16],
    [-0.0015, -0.0015, -0.0015, -0.0015, -0.0015, -0.0015, -0.0015, -0.0015, -0.0015]
).rename('SlopeBrown')


# Convert Tile Collection to List
tile_list = tiles_fc.toList(tiles_fc.size())

# 6. Define Number of Tiles
n_tiles = tile_list.length()

# 7. Define Analysis Period
start_year = 1986
end_year = 2024
year_list = ee.List.sequence(start_year, end_year)

#  8. Tile Processing Function
def process_tile(tile_index):
    tile_index = ee.Number(tile_index)
    tile_feature = ee.Feature(tile_list.get(tile_index))
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    #  8.1 Load Ecozones for this tile
    tile_ecozones = full_ecozone_fc.filterBounds(tile_geom)

    # 8.2 Rasterize Ecozone ID for this tile
    tile_ecozone_raster = tile_ecozones.reduceToImage(
        properties=['ID'],
        reducer=ee.Reducer.first()
    ).rename('Ecozone').clip(tile_geom)

    # 8.3 Build Per-Year Fire Memory Dictionary for This Tile
    fire_memory_dict = ee.Dictionary.fromLists(
        year_list.map(lambda year: ee.String(year)),
        year_list.map(lambda year: ee.ImageCollection([
            fire_raster.eq(ee.Number(year).subtract(offset)).selfMask()
            for offset in range(11)
        ]).max().clip(tile_geom))
    )

    return fire_memory_dict  # Optional, to confirm structure

# Confirm function output (optional, triggers server-side eval)
#print(process_tile(0).getInfo())


In [ ]:

# 4. Load Fire Raster
fire_raster = ee.Image("projects/sat-io/open-datasets/CA_FOREST/NBAC/NBAC_MRB_1972_to_2023")

# 5–7. Tile and Year Setup
tile_list = tiles_fc.toList(tiles_fc.size())
n_tiles = tile_list.length()
start_year = 1986
end_year = 2024
year_list = ee.List.sequence(start_year, end_year)

# 8. Process Each Tile
def process_tile(tile_index):
    tile_index = ee.Number(tile_index)
    tile_feature = ee.Feature(tile_list.get(tile_index))
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    tile_ecozone_raster = full_ecozone_fc.filterBounds(tile_geom).reduceToImage(
        ['ID'], ee.Reducer.first()
    ).rename('Ecozone').clip(tile_geom)

    def fire_memory(year):
        return ee.ImageCollection([fire_raster.eq(ee.Number(year).subtract(offset)).selfMask()
                                   for offset in range(11)]).max().clip(tile_geom)

    fire_memory_dict = ee.Dictionary.fromLists(
        year_list.map(lambda y: ee.String(y)),
        year_list.map(lambda y: fire_memory(y))
    )

    def load_and_score_landsat(year):
        start = ee.Date.fromYMD(year, 7, 1)
        end = ee.Date.fromYMD(year, 9, 30)

        def select_bands(col, bands_in, bands_out):
            return col.filterBounds(tile_geom).filterDate(start, end).select(bands_in, bands_out)

        l5 = select_bands(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2'),
                          ['SR_B1','SR_B3','SR_B4','SR_B5','SR_B7','QA_PIXEL','SR_ATMOS_OPACITY'],
                          ['BLUE','RED','NIR','SWIR1','SWIR2','QA','AERO'])

        l7 = select_bands(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'),
                          ['SR_B1','SR_B3','SR_B4','SR_B5','SR_B7','QA_PIXEL','SR_ATMOS_OPACITY'],
                          ['BLUE','RED','NIR','SWIR1','SWIR2','QA','AERO'])

        l8 = select_bands(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'),
                          ['SR_B2','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL','SR_QA_AEROSOL'],
                          ['BLUE','RED','NIR','SWIR1','SWIR2','QA','AERO'])

        l9 = select_bands(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2'),
                          ['SR_B2','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL','SR_QA_AEROSOL'],
                          ['BLUE','RED','NIR','SWIR1','SWIR2','QA','AERO'])

        collection = l5.merge(l7).merge(l8).merge(l9)

        def score_image(img):
            ndvi = img.normalizedDifference(['NIR', 'RED']).rename('NDVI')
            evi = img.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
                                 {'NIR': img.select('NIR'), 'RED': img.select('RED'), 'BLUE': img.select('BLUE')}).rename('EVI')
            nbr = img.normalizedDifference(['NIR', 'SWIR2']).rename('NBR')

            spacecraft = ee.String(img.get('SPACECRAFT_ID'))
            isL5 = spacecraft.compareTo(ee.String('LANDSAT_5')).eq(0)
            isL7 = spacecraft.compareTo(ee.String('LANDSAT_7')).eq(0)
            isL57 = isL5.Or(isL7)
            date = ee.Date(img.get('DATE_ACQUIRED'))
            isL7After2003 = isL7.And(date.millis().gt(ee.Date('2003-06-01').millis()))
            sensor_score = ee.Image.constant(1).where(isL7After2003, 0.5)
            slc_penalty = ee.Image.constant(1).where(isL7After2003, 0.8)

            cloud = img.select('QA').bitwiseAnd(1 << 3).neq(0)
            shadow = img.select('QA').bitwiseAnd(1 << 4).neq(0)
            cloud_shadow = cloud.Or(shadow)
            dist = cloud_shadow.updateMask(cloud_shadow).fastDistanceTransform(256).sqrt()
            dreq = ee.Number(50)
            dmid = dreq.divide(2)
            dist_score = ee.Image(1).divide(ee.Image(1).add(dist.min(dreq).subtract(dmid).multiply(-0.2).exp())).rename('DistScore')

            aero = img.select('AERO')
            aot = ee.Image(ee.Algorithms.If(
                isL57, aero.multiply(0.001),
                aero.bitwiseAnd(192).rightShift(6).expression(
                    "b(0)==0?0.05:b(0)==1?0.1:b(0)==2?0.2:b(0)==3?0.3:0",
                    {'b(0)': aero.bitwiseAnd(192).rightShift(6)}))
            ).rename('AOT_Debug')

            Omin, Omax = ee.Number(0.2), ee.Number(0.3)
            Omid = Omin.add(Omax).divide(2)
            opacity_score = ee.Image(1).subtract(
                ee.Image(1).divide(ee.Image(1).add(aot.min(Omax).subtract(Omid).multiply(-4).exp()))
            ).where(aot.lte(Omin), 1).where(aot.gte(Omax), 0).rename('OpacityScore')

            doy = ee.Number(date.getRelative('day', 'year'))
            mu, sigma = ee.Number(213), ee.Number(38)
            norm = sigma.multiply((2 * 3.14159)**0.5)
            max_score = ee.Number(1).divide(norm)
            raw = doy.subtract(mu).pow(2).divide(sigma.pow(2)).multiply(-0.5).exp()
            doy_score = raw.divide(norm).divide(max_score)
            doy_score_img = ee.Image.constant(doy_score).toFloat().rename('DOYScore')

            total_score = sensor_score.multiply(slc_penalty).add(dist_score).add(opacity_score).add(doy_score_img).rename('TotalScore')

            return img.addBands([ndvi, evi, nbr, dist_score, opacity_score, doy_score_img, total_score])

        return collection.map(score_image)

    def get_annual_best(year):
        scored = load_and_score_landsat(year)
        best = scored.qualityMosaic('TotalScore').clip(tile_geom)
        return best.select(['NDVI', 'EVI', 'NBR']) \
                   .set('year', year) \
                   .set('tile_id', tile_id) \
                   .set('system:time_start', ee.Date.fromYMD(year, 7, 1).millis())

    annual_best = ee.ImageCollection(year_list.map(get_annual_best))

    def smooth_collection(coll):
        return ee.ImageCollection(year_list.map(
            lambda y: coll.filter(ee.Filter.eq('year', y)).first()
                      .set('year', y)
                      .set('system:time_start', ee.Date.fromYMD(y, 7, 1).millis())
        ))

    return smooth_collection(annual_best)

#  Build Full Smoothed Collection from All Tiles
tile_indices = ee.List.sequence(0, n_tiles.subtract(1))

image_lists = tile_indices.map(lambda i: process_tile(i).toList(process_tile(i).size()))
flattened_images = image_lists.flatten()

#Before collecting the into the collection below other post-processing were performed,
#including despiking, multi-factor wieghted infiling, etc {full details can be provided upon reasonable request}

full_smoothed_collection = ee.ImageCollection.fromImages(flattened_images)

print(" Full Smoothed ImageCollection Ready")


In [ ]:
#  9.1 Compute Trend & Base + Z-Significant Classification Per Tile + Return SenSlope & ZScore

def compute_veg_classification(tile_feature):
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    # Filter smoothed collection for this tile
    tile_col = full_smoothed_collection.filter(ee.Filter.eq('tile_id', tile_id))
    years = ee.List(tile_col.aggregate_array('year')).sort()

    # Build ImageCollection for trend analysis
    def build_trend_image(y):
        y = ee.Number(y)
        img = tile_col.filter(ee.Filter.eq('year', y)).first()
        rel_year = y.subtract(start_year)
        return img.select('NDVI').addBands(
            ee.Image.constant(rel_year).rename('relativeYear').toFloat()
        )
    imgs_for_trend = ee.ImageCollection(years.map(build_trend_image))

    # Sen's Slope and Kendall's Tau
    sen = imgs_for_trend.select(['relativeYear', 'NDVI']) \
        .reduce(ee.Reducer.sensSlope()).select('slope').rename('SenSlope')

    tau = imgs_for_trend.select(['relativeYear', 'NDVI']) \
        .reduce(ee.Reducer.kendallsCorrelation()).select('NDVI_tau').rename('KendallTau')

    N = ee.Number(end_year).subtract(start_year).add(1)
    z_score = tau.multiply(3).multiply(N.multiply(N.subtract(1)).sqrt()) \
        .divide(ee.Number(2).multiply(N.multiply(2).add(5).sqrt())) \
        .rename('KendallZ')

    # Threshold-based classifications
    greening = sen.gt(0.0015).And(z_score.abs().gt(0.52))
    browning = sen.lt(-0.0015).And(z_score.abs().gt(0.52))
    stable = sen.gte(-0.0015).And(sen.lte(0.0015)).And(z_score.abs().gt(0.52))

    first_ndvi = tile_col.sort('year').first().select('NDVI')
    last_ndvi = tile_col.sort('year', False).first().select('NDVI')
    ndvi_drop = last_ndvi.subtract(first_ndvi)
    persistent_decline = ndvi_drop.lt(-0.0015)

    greening_class = greening.And(slope_green_image.clip(tile_geom).gte(0.0015))
    browning_class = browning.And(
        slope_brown_image.clip(tile_geom).lte(-0.0015)
    ).And(persistent_decline)
    stable_class = stable.And(
        slope_green_image.clip(tile_geom).lte(0.0015)
    ).And(
        slope_brown_image.clip(tile_geom).gte(-0.0015)
    )

    veg_class = ee.Image(0) \
        .where(greening_class, 1) \
        .where(browning_class, 2) \
        .where(stable_class, 3) \
        .rename('VegChangeClass') \
        .set('tile_id', tile_id)

    #  Z-Score based classification
    significant = z_score.abs().gte(1.62)

    ndvi_trend_zsen = ee.Image(0) \
        .where(significant.And(sen.lt(-0.003)), 1) \
        .where(significant.And(sen.gte(-0.003).And(sen.lt(0))), 2) \
        .where(significant.And(sen.eq(0)), 3) \
        .where(significant.And(sen.gt(0).And(sen.lte(0.003))), 4) \
        .where(significant.And(sen.gt(0.003)), 5) \
        .rename('NDVI_Trend_ZSen')

    return veg_class \
        .addBands(ndvi_trend_zsen) \
        .addBands(sen) \
        .addBands(z_score) \
        .set('tile_id', tile_id) \
        .set('system:time_start', ee.Date.fromYMD(end_year, 7, 1).millis()) \
        .clip(tile_geom)


In [ ]:
# --- Export NDVI KendallZ per tile (Drive) -------------------------------
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()

crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]  # ~30 m

for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    try:
        tile_id_val = tile.get('tile_id').getInfo()
    except Exception:
        tile_id_val = i + 1

    # Get the raw Kendall Z band from your function
    ndvi_kz_tile = compute_veg_classification(tile).select('KendallZ')

    # Export as real NoData (no unmask, no updateMask)
    image_to_export = ndvi_kz_tile.toFloat().clip(tile_geom)

    desc  = f'NDVI_KendallZ_{start_year}_{end_year}_{tile_id_val}'
    fname = f'NDVI_KendallZ_{start_year}_{end_year}_{tile_id_val}'

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=desc,
        folder='NDVI_KendallZ_30m',
        fileNamePrefix=fname,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()
    print(f" Export task submitted for tile {tile_id_val}: {desc}")

print("All export tasks submitted.")


In [ ]:
import ee
import geemap

#  Initialize Earth Engine
ee.Initialize()

#  Load tiles and NDVI mosaic for 1986
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()

# NDVI 1986 mosaic (previously generated)
ndvi_1986 = full_smoothed_collection \
    .filter(ee.Filter.eq('year', 1986)) \
    .select('NDVI') \
    .mosaic() \
    .clip(tiles_fc.geometry())

# 🛠 Export settings
no_data_value = 9999
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]  # ≈30m

#  Loop through each tile and export
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1

    description = f'NDVI_1986_{tile_id}'
    file_name_prefix = f'NDVI_1986_{tile_id}'

    image_to_export = ndvi_1986.unmask(no_data_value).toFloat().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='Starting_NDVI_30_1',
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print("All export tasks submitted. Go to the Earth Engine Code Editor 'Tasks' tab to run them manually.")


In [ ]:
by_tile_veg_class = ee.ImageCollection(tile_list.map(lambda f: compute_veg_classification(f)))

In [ ]:
base_veg_class = by_tile_veg_class.select('VegChangeClass').mosaic()
ndvi_trend_zsen = by_tile_veg_class.select('NDVI_Trend_ZSen').mosaic()
static_sen_slope = by_tile_veg_class.select('SenSlope').mosaic()
static_z_score = by_tile_veg_class.select('KendallZ').mosaic()


In [ ]:
def process_tile_with_fire_correction(tile_index):
    tile_index = ee.Number(tile_index)
    tile_feature = tile_list.get(tile_index)  #  Correct: No need to wrap with ee.Feature
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    fire_occurrence = fire_raster.gte(1986).selfMask().clip(tile_geom)
    fire_year = fire_raster.clip(tile_geom).rename('fireYear')

    def build_img(y):
        y = ee.Number(y)
        img = full_smoothed_collection \
            .filter(ee.Filter.eq('year', y)) \
            .filter(ee.Filter.eq('tile_id', tile_id)).first()
        rel_year = y.subtract(start_year)
        return img.select('NDVI') \
            .addBands(ee.Image.constant(y).rename('absoluteYear').toFloat()) \
            .addBands(ee.Image.constant(rel_year).rename('relativeYear').toFloat()) \
            .clip(tile_geom)

    imgs = ee.ImageCollection(year_list.map(build_img))

    pre_fire = imgs.map(lambda img: img.updateMask(img.select('absoluteYear').lt(fire_year))) \
        .filter(ee.Filter.listContains('system:band_names', 'NDVI'))

    post_fire = imgs.map(lambda img: img.updateMask(img.select('absoluteYear').gt(fire_year))) \
        .filter(ee.Filter.listContains('system:band_names', 'NDVI'))

    pre_count = pre_fire.size()
    post_count = post_fire.size()

    # Pre-fire slope & Z
    pre_sen = pre_fire.select(['relativeYear', 'NDVI']).reduce(ee.Reducer.sensSlope()).select('slope')
    pre_tau = pre_fire.select(['relativeYear', 'NDVI']).reduce(ee.Reducer.kendallsCorrelation()).select('NDVI_tau')
    pre_z = pre_tau.multiply(3).multiply(pre_count.multiply(pre_count.subtract(1)).sqrt()) \
        .divide(ee.Number(2).multiply(pre_count.multiply(2).add(5)).sqrt()).rename('PreFire_Zscore')

    # Post-fire slope & Z
    post_sen = post_fire.select(['relativeYear', 'NDVI']).reduce(ee.Reducer.sensSlope()).select('slope')
    post_tau = post_fire.select(['relativeYear', 'NDVI']).reduce(ee.Reducer.kendallsCorrelation()).select('NDVI_tau')
    post_z = post_tau.multiply(3).multiply(post_count.multiply(post_count.subtract(1)).sqrt()) \
        .divide(ee.Number(2).multiply(post_count.multiply(2).add(5)).sqrt()).rename('PostFire_Zscore')

    pre_sig = pre_z.abs().gt(0.52)
    post_sig = post_z.abs().gt(0.52)
    both_sig = pre_sig.And(post_sig)

    weight_total = pre_count.add(post_count)
    final_slope = pre_sen.multiply(pre_count).add(post_sen.multiply(post_count)) \
        .divide(weight_total).rename('FireAdjusted_Slope').updateMask(both_sig)

    green = final_slope.gte(0.0015)
    brown = final_slope.lte(-0.0015)
    stable = final_slope.gt(-0.005).And(final_slope.lte(0.0015))

    fire_correction = ee.Image(0) \
        .where(green, 1) \
        .where(brown, 2) \
        .where(stable, 3) \
        .updateMask(both_sig.And(fire_occurrence))

    corrected_class = base_veg_class.where(fire_correction.neq(0), fire_correction).rename('Final_VegChange')

    trend_class = ee.Image(0) \
        .where(final_slope.lt(-0.003), 1) \
        .where(final_slope.gte(-0.003).And(final_slope.lt(0)), 2) \
        .where(final_slope.eq(0), 3) \
        .where(final_slope.gt(0).And(final_slope.lte(0.003)), 4) \
        .where(final_slope.gt(0.003), 5) \
        .rename('FireAdjusted_TrendClass')

    return final_slope.rename('FireAdjusted_Slope') \
        .addBands(pre_sen.rename('PreFire_Slope')) \
        .addBands(post_sen.rename('PostFire_Slope')) \
        .addBands(pre_z) \
        .addBands(post_z) \
        .addBands(corrected_class) \
        .addBands(trend_class) \
        .set('tile_id', tile_id) \
        .set('system:time_start', ee.Date.fromYMD(end_year, 7, 1).millis()) \
        .clip(tile_geom)


In [ ]:
indices = ee.List.sequence(0, tile_list.size().subtract(1))
fire_corrected_tiles = ee.ImageCollection(
    indices.map(lambda i: process_tile_with_fire_correction(i))
)


In [ ]:
fire_adjusted_slope = fire_corrected_tiles.select('FireAdjusted_Slope').mosaic()
pre_slope = fire_corrected_tiles.select('PreFire_Slope').mosaic()
post_slope = fire_corrected_tiles.select('PostFire_Slope').mosaic()
pre_z = fire_corrected_tiles.select('PreFire_Zscore').mosaic()
post_z = fire_corrected_tiles.select('PostFire_Zscore').mosaic()
fire_corrected_veg = fire_corrected_tiles.select('Final_VegChange').mosaic()
reclassified_trend = fire_corrected_tiles.select('FireAdjusted_TrendClass').mosaic()


In [ ]:
fire_occurrence = fire_raster.gte(1986).selfMask()
final_static_slope = static_sen_slope.where(
    fire_occurrence, fire_adjusted_slope
).rename('Final_Static_SenSlope')


In [ ]:
# Boolean mask: True where |Z| = 0.84 (non-significant)
non_significant_bool = static_z_score.abs().lt(0.84).rename('ZScore_NonSignificant')

In [ ]:
#  Boolean mask where absolute Z-score < 0.84
zscore_nonsig_mask = static_z_score.abs().lt(0.84).selfMask()

In [ ]:
initial_lineartrend_class = ee.Image(0) \
    .where(final_static_slope.lte(-0.003), 1) \
    .where(final_static_slope.gt(-0.003).And(final_static_slope.lte(-0.0015)), 2) \
    .where(final_static_slope.gt(-0.0015).And(final_static_slope.lt(0.0015)), 3) \
    .where(final_static_slope.gte(0.0015).And(final_static_slope.lt(0.003)), 4) \
    .where(final_static_slope.gte(0.003), 5)


In [ ]:
#building of masks to filter the trend grids
#  1. Load region of interest
region = ee.FeatureCollection("projects/danielsiglab/assets/studyarea50")
region_geom = region.geometry()

#  2. Define years and remapping
years = list(range(1984, 2023))
original_classes = [0, 20, 31, 32, 33, 40, 50, 80, 81, 100, 210, 220, 230]
remap_classes = list(range(13))
mask_classes = [1, 2]

# 3. Get reclassified image per year
def get_reclass_lc(year):
    img = ee.ImageCollection("projects/sat-io/open-datasets/CA_FOREST_LC_VLCE2") \
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .first() \
        .select('b1') \
        .remap(original_classes, remap_classes) \
        .rename('LC') \
        .set('year', year)
    return img

#  Reclassify and build collection
lc_collection = ee.ImageCollection([get_reclass_lc(y) for y in years])

# Create persistent water/snow mask
mask_collection = lc_collection.map(lambda img: img.remap(mask_classes, [1, 1], 0).rename('Mask'))
sum_mask = mask_collection.reduce(ee.Reducer.sum())
persistent_mask = sum_mask.eq(len(years)).rename('Persistent_Mask')
final_water_snow_mask = persistent_mask.updateMask(persistent_mask.neq(0)).unmask(0).rename('Persistent_Mask')

# Load built-up surface for 2020 and clip
built_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')
built_clipped = built_2020.clip(region_geom)
built_mask = built_clipped.gt(0).rename('built_mask')

#  Combine masks
combined_mask = final_water_snow_mask.max(built_mask).rename('Combined_Mask')

# Export to asset
#task = ee.batch.Export.image.toAsset(
    #image=combined_mask.clip(region_geom),
    #description='Export_Combined_Mask_BuiltUp_WaterSnow',
    #assetId='projects/danielsiglab/assets/combined_builtup_water_snow_mask',
    #region=region_geom,
    #scale=30,
    #maxPixels=1e13
#)
#task.start()
#print("Export task started. Monitor it from the Earth Engine Tasks tab.")


In [ ]:
# Reclassify lineartrend_class where combined_mask == 1 to value 6
final_lineartrend_class = initial_lineartrend_class.where(combined_mask.eq(1), 6).rename('LinearTrend_Class_Corrected')


In [ ]:
# Create binary ecozone raster (1 = inside, 0 = outside)
ecozone_mask = ee.Image.constant(1).clip(full_ecozone_fc).unmask(0).rename('Ecozone_Mask')

In [ ]:
# Step 2: Reclassify linear trend class using the ecozone mask
final_class2 = final_lineartrend_class \
    .where(ecozone_mask.eq(0), 7) \
    .rename('LinearTrend_Class_Corrected_Final')

In [ ]:
#Get the unioned geometry of all tiles
tiles_extent = tiles_fc.geometry()

# Step 2: Clip the final class image to the tile extent
final_class2_clipped = final_class2.clip(tiles_extent)


In [ ]:
#To export the linear class properly thresholded

In [ ]:
#  Load final classified image (assumed already computed)
# Load tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()  # Convert to client-side integer

# Export settings
no_data_value = 9999
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

#  Loop through tiles and create export tasks
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1  # Sequential numbering from 1

    description = f'linearclass_{tile_id}'
    file_name_prefix = f'linearclass_{tile_id}'

    image_to_export = final_class2_clipped.unmask(no_data_value).toInt16().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='LINEAR_TREND',  # Folder name updated
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print("All export tasks submitted. Go to the Earth Engine Code Editor 'Tasks' tab to run them manually.")


In [ ]:
#To export the linear slope grid

In [ ]:
#  Load tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()

#  Export settings
no_data_value = 0  # Suitable for float rasters
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

# Loop through tiles and export slope
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1

    description = f'linearslope_{tile_id}'
    file_name_prefix = f'linearslope_{tile_id}'

    image_to_export = final_static_slope_clipped.unmask(no_data_value).toFloat().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='linear_slope',  # Folder name for slope
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print("All slope export tasks submitted. Check the Earth Engine 'Tasks' tab to monitor.")


In [ ]:
#To fit the exponential trend model

In [ ]:
import ee

#  10.6 Tile-Based Static Exponential Growth Fitting (Per Pixel)
def per_tile_exp_fit(tile_index):
    tile_index = ee.Number(tile_index)
    tile_feature = ee.Feature(tile_list.get(tile_index))
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    # Filter the smoothed NDVI collection for this tile
    tile_col = full_smoothed_collection.filter(ee.Filter.eq('tile_id', tile_id))
    years = ee.List(tile_col.aggregate_array('year')).sort()

    # Build image collection with log(NDVI) and relative year
    def build_log_img(y):
        y = ee.Number(y)
        img = tile_col.filter(ee.Filter.eq('year', y)).first()
        rel_year = ee.Image.constant(y.subtract(start_year)).rename('relativeYear').toFloat()
        log_ndvi = img.select('NDVI').log().rename('log_NDVI')
        return log_ndvi.addBands(rel_year).clip(tile_geom)

    imgs_for_fit = ee.ImageCollection(years.map(build_log_img))

    # Linear fit in log-space: log(NDVI) = b * t + log(a)
    fit = imgs_for_fit.select(['relativeYear', 'log_NDVI']).reduce(ee.Reducer.linearFit())
    slope = fit.select('scale').rename('Exp_Slope')              # b
    intercept = fit.select('offset').rename('Exp_Intercept')     # log(a)

    # Back-transform slope to approximate per-year growth rate: exp(b) - 1
    transformed_slope = slope.exp().subtract(1).rename('Exp_Slope_Transformed')

    # Compute MSE (Mean Squared Error) in original NDVI space
    def compute_sq_error(img):
        obs = img.select('log_NDVI').exp()
        pred = intercept.add(slope.multiply(img.select('relativeYear'))).exp()
        return obs.subtract(pred).pow(2).rename('SqError')

    mse = ee.ImageCollection(imgs_for_fit.map(compute_sq_error)).mean().rename('Exp_RMSE')

    # Compute TSS (Total Sum of Squares)
    mean_ndvi = tile_col.select('NDVI').mean()
    def tss_img(img):
        obs = img.select('NDVI')
        return obs.subtract(mean_ndvi).pow(2).rename('TSS')

    tss_mean = ee.ImageCollection(tile_col.map(tss_img)).mean().rename('TSS_Mean')

    # Compute R² = 1 - (MSE / TSS)
    r_squared = ee.Image(1).subtract(mse.divide(tss_mean)).rename('Exp_R2')

    # Return all bands for the tile
    return transformed_slope.addBands(intercept).addBands(r_squared).addBands(mse) \
        .clip(tile_geom).set('tile_id', tile_id)

# Apply exponential fitting to all tiles
exp_fit_tiles = ee.ImageCollection.fromImages(
    ee.List.sequence(0, n_tiles.subtract(1)).map(per_tile_exp_fit)
)

# Merge into one per-pixel layer
per_pixel_exp_stack = exp_fit_tiles.mosaic()



In [ ]:
# Select R² from exponential and logistic fits
exp_r2 = per_pixel_exp_stack.select('Exp_R2')
# Mask pixels with Exp_R2 ≥ 0.5
exp_significant = exp_r2.gte(0.6).selfMask()

# Add layers to the map
Map.addLayer(exp_r2, {
    'min': 0, 'max': 1,
    'palette': ['white', 'limegreen']
}, 'Exponential R²')

Map.addLayer(exp_significant, {
    'min': 0, 'max': 1,
    'palette': ['red']
}, 'Exp R² ≥ 0.5')

Map

In [ ]:
# Step 10.8 Per-Pixel Logistic NDVI Fitting with pImage as c (Relative Year Adjustment)

def logistic_fit_tile(tile_index):
    tile_feature = ee.Feature(tile_list.get(tile_index))
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.get('tile_id')

    tile_col = full_smoothed_collection.filter(ee.Filter.eq('tile_id', tile_id))
    years = ee.List(tile_col.aggregate_array('year')).sort()

    ndvi_min = tile_col.select('NDVI').reduce(ee.Reducer.min())
    ndvi_max = tile_col.select('NDVI').reduce(ee.Reducer.max())

    p_image_tile = pimage.clip(tile_geom)
    c_image = p_image_tile.subtract(start_year).rename('c').toFloat()

    norm_ndvi = tile_col.map(lambda img:
        img.select('NDVI').subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).rename('Norm_NDVI')
        .addBands(ee.Image.constant(ee.Number(img.get('year')).subtract(start_year)).rename('relYear').toFloat())
        .clip(tile_geom)
    )

    logit_collection = norm_ndvi.map(lambda img:
        img.select('Norm_NDVI').divide(ee.Image(1).subtract(img.select('Norm_NDVI'))).log().rename('Logit_NDVI')
        .addBands(img.select('relYear'))
    )

    fit = logit_collection.select(['relYear', 'Logit_NDVI']).reduce(ee.Reducer.linearFit())
    b = fit.select('scale').rename('b')
    d = fit.select('offset').rename('d')

    log_slope_trans = b.exp().subtract(1).rename('Log_Slope_Transformed')
    a = ndvi_max.subtract(ndvi_min).rename('a')

    mse = ee.ImageCollection(norm_ndvi.map(lambda img: (
        img.select('Norm_NDVI')
        .subtract(a.divide(ee.Image(1).add(b.multiply(img.select('relYear').subtract(c_image)).multiply(-1).exp())).add(d))
        .pow(2).rename('SqError')
    ))).mean().rename('Logistic_RMSE')

    mean_obs = norm_ndvi.select('Norm_NDVI').mean()
    tss = norm_ndvi.map(lambda img: img.select('Norm_NDVI').subtract(mean_obs).pow(2).rename('TSS'))
    tss_mean = ee.ImageCollection(tss).mean().rename('TSS_Mean')

    r2 = ee.Image(1).subtract(mse.divide(tss_mean)).rename('Logistic_R2')

    return log_slope_trans.addBands(c_image).addBands(r2).addBands(mse).addBands(a).addBands(d).clip(tile_geom).set('tile_id', tile_id)

# Run across all tiles
logistic_fit_tiles = ee.ImageCollection.fromImages(
    ee.List.sequence(0, n_tiles.subtract(1)).map(lambda i: logistic_fit_tile(i))
)

# Mosaic results
per_pixel_logistic_stack = logistic_fit_tiles.mosaic()

Map.addLayer(per_pixel_logistic_stack.select('Log_Slope_Transformed'),
             {'min': -0.01, 'max': 0.01, 'palette': ['blue', 'white', 'orange']},
             'Logistic Slope Transformed')
print('Logistic Fitting with Slope Transformation Completed.')



In [ ]:
exp_significant = per_pixel_exp_stack.select('Exp_R2').gte(0.6)


In [ ]:
exp_slope = per_pixel_exp_stack.select('Exp_Slope_Transformed')

exp_class = ee.Image(0) \
    .where(exp_slope.lte(-0.002), 1) \
    .where(exp_slope.gt(-0.002).And(exp_slope.lte(-0.00132)), 2) \
    .where(exp_slope.gt(-0.00132).And(exp_slope.lt(0.00132)), 3) \
    .where(exp_slope.gte(0.00132).And(exp_slope.lt(0.002)), 4) \
    .where(exp_slope.gte(0.002), 5)


In [ ]:
exp_class_final = exp_class.where(exp_significant.Not(), 0).rename('Exp_Trend_Class')


In [ ]:

# Step 1: Compute the boolean mask: where linear trend is 0 and exponential class is > 0
new_exp_only = lineartrend_class.eq(0).And(exp_class_final.gt(0))

# Step 2: Clip it to full_ecozone_fc and convert to 0/1 values
new_exp_only_clipped = new_exp_only.clip(full_ecozone_fc).unmask(0).int8().rename('Exp_Only_Clipped')


In [ ]:
exp_stack = per_pixel_exp_stack
exp_r2 = exp_stack.select('Exp_R2')
exp_slope = exp_stack.select('Exp_Slope_Transformed')


In [ ]:
exp_significant = exp_r2.gte(0.6)


In [ ]:
linear_no_trend = lineartrend_class.eq(0)


In [ ]:
condition_1 = linear_no_trend.And(exp_significant)


In [ ]:
initial_ndvi = full_smoothed_collection.filter(ee.Filter.eq('year', 1986)).first().select('NDVI')


In [ ]:
years = ee.Number(2023).subtract(1986)

# Linear NDVI change over 37 years
linear_change = final_static_slope.multiply(years)

# Exponential NDVI change: ΔNDVI = NDVI₀ * [(1 + r)^t - 1]
exp_multiplier = exp_slope.add(1).pow(years)
exp_change = initial_ndvi.multiply(exp_multiplier.subtract(1))

# Condition 2: where exponential change is significantly larger than linear
condition_2 = exp_change.subtract(linear_change).gt(0.02)


In [ ]:
use_exponential = condition_1.Or(condition_2)


In [ ]:
log_slope = per_pixel_logistic_stack.select('Log_Slope_Transformed')

log_class = ee.Image(0) \
    .where(log_slope.lte(-0.002), 1) \
    .where(log_slope.gt(-0.002).And(log_slope.lte(-0.00132)), 2) \
    .where(log_slope.gt(-0.00132).And(log_slope.lt(0.00132)), 3) \
    .where(log_slope.gte(0.00132).And(log_slope.lt(0.002)), 4) \
    .where(log_slope.gte(0.002), 5)


In [ ]:
# Extract the R² band from the logistic fit stack
logistic_r2 = per_pixel_logistic_stack.select('Logistic_R2')

# Create a significance mask: True where R² ≥ 0.5
logistic_significant = logistic_r2.gte(0.6).rename('Logistic_Significant')


In [ ]:
log_class_final = log_class.where(logistic_significant.Not(), 0).rename('Log_Trend_Class')


In [ ]:
import geemap

# Clip the exponential slope classification to the ecozone extent
log_class_clipped = log_class_final.clip(full_ecozone_fc)

#  Define the visualization parameters using the same palette as `lineartrend_vis`
exptrend_vis = {
    'min': 0,
    'max': 5,
    'palette': [
        '#A9A9A9',  # Gray - No significant trend
        '#8B4513',  # SaddleBrown - Strong negative
        '#DAA520',  # GoldenRod - Moderate negative
        '#4682B4',  # SteelBlue - Near stable
        '#90EE90',  # LightGreen - Moderate positive
        '#006400'   # DarkGreen - Strong positive
    ]
}

#  Create a new map instance to avoid overlap
logSlopeMap = geemap.Map()

# Add the clipped classification layer
logSlopeMap.addLayer(log_class_clipped, exptrend_vis, ' log Slope Class (Clipped & Colored)')

#Display the new standalone map
logSlopeMap



In [ ]:
# Step 1: Compute NDVI baseline and final means
ndvi_early = full_smoothed_collection \
    .filter(ee.Filter.inList('year', [1986, 1987, 1988])) \
    .select('NDVI') \
    .mean().rename('NDVI_Early')

ndvi_late = full_smoothed_collection \
    .filter(ee.Filter.inList('year', [2021, 2022, 2023])) \
    .select('NDVI') \
    .mean().rename('NDVI_Late')

observed_change = ndvi_late.subtract(ndvi_early)

# Compute modeled NDVI changes
years_diff = ee.Number(2023 - 1986)

# Linear model NDVI estimate
linear_slope = final_static_slope
linear_predicted_change = linear_slope.multiply(years_diff)
ndvi_linear_predicted = ndvi_early.add(linear_predicted_change)

# Exponential model NDVI estimate
exp_r2 = per_pixel_exp_stack.select('Exp_R2')
exp_slope = per_pixel_exp_stack.select('Exp_Slope_Transformed')
exp_significant = exp_r2.gte(0.6)

exp_multiplier = exp_slope.add(1).pow(years_diff)
ndvi_exp_predicted = ndvi_early.multiply(exp_multiplier)

# Step 3: Compare models to observed NDVI
diff_linear = ndvi_linear_predicted.subtract(ndvi_late).abs()
diff_exp = ndvi_exp_predicted.subtract(ndvi_late).abs()

# Select pixels where exponential is a better fit
exp_better_fit = diff_exp.lt(diff_linear).And(exp_significant)

# Also include where linear had no trend (class 0)
linear_no_trend = lineartrend_class.eq(0)
combined_condition = linear_no_trend.Or(exp_better_fit)

# Update final class and/or slope map
updated_trend_class = lineartrend_class.where(combined_condition, exp_class_final)
updated_slope = final_static_slope.where(combined_condition, exp_slope)

# Optional: visualize or export
#Map.addLayer(combined_condition.selfMask(), {'palette': ['orange']}, 'Pixels better explained by Exp')
Map.addLayer(combined_condition, {'min': 0, 'max': 8}, 'Updated Trend Class')
Map

In [ ]:
# Define visualization style for boolean mask
combined_vis = {
    'min': 0,
    'max': 1,
    'palette': ['gray', 'orange']  # 0 = keep linear, 1 = use exponential
}

#  Create a new map to isolate this visualization
combinedMap = geemap.Map()

#  Add the combined condition grid
combinedMap.addLayer(combined_condition, combined_vis, 'Better Explained by Exp or No Trend')

#  Display
combinedMap


In [ ]:
#  Create a new map instance to isolate exponential-better-only pixels
expBetterOnlyMap = geemap.Map(center=[60, -96], zoom=3, height="1000px")

#  Clip to ecozone boundaries to match styling
exp_better_masked_full = exp_better_fit.clip(full_ecozone_fc)

#  Define two-class visualization style: 0 = linear better, 1 = exp better
exp_better_vis = {
    'min': 0,
    'max': 1,
    'palette': ['#D3D3D3', 'red']  # light gray for linear, red for exponential
}

# Add to map
expBetterOnlyMap.addLayer(exp_better_masked_full, exp_better_vis, ' Exp vs Linear Fit Comparison')

# Add boundaries
expBetterOnlyMap.addLayer(
    full_ecozone_fc.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Ecozone Boundary'
)

canada_country = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq("country_na", "Canada"))
canada_provinces = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "Canada"))

expBetterOnlyMap.addLayer(
    canada_country.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Canada Border'
)

expBetterOnlyMap.addLayer(
    canada_provinces.style(**{'color': 'gray', 'fillColor': '00000000', 'width': 1}),
    {}, 'Province Boundaries'
)

# Add a scale bar
from ipyleaflet import ScaleControl
expBetterOnlyMap.add_control(ScaleControl(position='bottomleft'))

#  Add updated legend for Exp vs Linear better fit
from ipywidgets import HTML
legend_html = HTML(
    value="""
    <div style='background:white; font-size:16px; padding:10px; border:none; margin-left:40px;'>
        <b>Pixel Best Fit </b><br>
        <div><span style='background:#D3D3D3; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Linear</div>
        <div><span style='background:red; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Exponential</div>
    </div>
    """
)

from ipyleaflet import WidgetControl
expBetterOnlyMap.add_control(WidgetControl(widget=legend_html, position='bottomleft'))

# Display the map
expBetterOnlyMap


In [ ]:
# Step 6: Convert exp_better_fit (boolean) to class values
# 1 = exponential better fit, 0 = linear better
exp_better_class = exp_better_fit.where(exp_better_fit, 1).unmask(0).uint8()

# Step 7: Identify pixels where exponential recovered NDVI in linear class 0
# => exponential R² ≥ 0.5 and linear model originally showed no trend
exp_class2_mask = linear_no_trend.And(exp_significant)

# Step 8: Assign class 2 where exponential recovered linear class 0
# Only replace pixels originally class 0 (i.e., where exp_better_class == 0)
exp_3class_grid = exp_better_class.where(
    exp_better_class.eq(0).And(exp_class2_mask),
    2
).uint8()


In [ ]:
print("Bands in exp_better_fit:", exp_better_fit.bandNames().getInfo())
print("Bands in exp_significant:", exp_significant.bandNames().getInfo())


In [ ]:
# Define visualization style for the 3-class grid
# 0 = Linear better, 1 = Exp better, 2 = Exp recovered from linear 0
exp_3class_vis = {
    'min': 0,
    'max': 2,
    'palette': ['#D3D3D3', '#FF0000', '#32CD32']  # gray, red, lime green
}

# Create a new map instance
exp3classMap = geemap.Map(center=[60, -96], zoom=3, height="1000px")

# Clip to ecozone boundaries for styling consistency
exp_3class_masked = exp_3class_grid.clip(full_ecozone_fc)

# Add the 3-class grid to the map
exp3classMap.addLayer(exp_3class_masked, exp_3class_vis, '3-Class Exp vs Linear Fit')

#  Add ecozone and national boundaries
exp3classMap.addLayer(
    full_ecozone_fc.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Ecozone Boundary'
)
exp3classMap.addLayer(
    canada_country.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Canada Border'
)
exp3classMap.addLayer(
    canada_provinces.style(**{'color': 'gray', 'fillColor': '00000000', 'width': 1}),
    {}, 'Province Boundaries'
)

# Add legend for 3-class interpretation
legend_html_3class = HTML(
    value="""
    <div style='background:white; font-size:16px; padding:10px; border:none; margin-left:40px;'>
        <b>NDVI Model Fit Classification</b><br>
        <div><span style='background:#D3D3D3; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Linear Fit Better (0)</div>
        <div><span style='background:#FF0000; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Exponential Fit Better (1)</div>
        <div><span style='background:#32CD32; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Exponential Recovered Linear No Trend (2)</div>
    </div>
    """
)
exp3classMap.add_control(WidgetControl(widget=legend_html_3class, position='bottomleft'))

#  Add scale control
exp3classMap.add_control(ScaleControl(position='bottomleft'))

#  Display the map
exp3classMap


In [ ]:
# Step 8: Identify remaining pixels in exp_3class_grid that are still 0 (Linear better)
still_linear_pixels = exp_3class_grid.eq(0)

# Step 9: Identify where original linear trend was non-zero
non_zero_linear = lineartrend_class.gt(0)

# Step 10: Find pixels where linear was non-zero but exp_3class still says 0
linear_only_fill = still_linear_pixels.And(non_zero_linear)

# Step 11: Update exp_3class_grid to assign class 3 at those positions
exp4class = exp_3class_grid.where(linear_only_fill, 3)


In [ ]:
# Create a new map to visualize exp4class
exp4Map = geemap.Map(center=[60, -96], zoom=3, height="1000px")

# Define visualization for exp4class: 0=insignificant, 1=exp better, 2=newly captured, 3=linear trend
exp4_vis = {
    'min': 0,
    'max': 3,
    'palette': ['#D3D3D3', '#FF0000', '#FFA500', '#0000FF']  # gray, red, orange, blue
}

#  Add exp4class layer
exp4Map.addLayer(exp4class.clip(full_ecozone_fc), exp4_vis, 'Full Model Fit Comparison (4 classes)')

#  Add ecozone & national boundaries
exp4Map.addLayer(
    full_ecozone_fc.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Ecozone Boundary'
)

canada_country = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq("country_na", "Canada"))
canada_provinces = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "Canada"))

exp4Map.addLayer(
    canada_country.style(**{'color': 'black', 'fillColor': '00000000', 'width': 1}),
    {}, 'Canada Border'
)

exp4Map.addLayer(
    canada_provinces.style(**{'color': 'gray', 'fillColor': '00000000', 'width': 1}),
    {}, 'Province Boundaries'
)

# 📏 Add scale bar
from ipyleaflet import ScaleControl
exp4Map.add_control(ScaleControl(position='bottomleft'))

#  Add legend with 4 class labels
from ipywidgets import HTML
legend_html_exp4 = HTML(
    value="""
    <div style='background:white; font-size:16px; padding:10px; border:none; margin-left:40px;'>
        <b>NDVI Model Fit (4-Class)</b><br>
        <div><span style='background:#D3D3D3; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Insignificant (0)</div>
        <div><span style='background:#FF0000; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Exponential Better (1)</div>
        <div><span style='background:#FFA500; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>New Exponential Capture (2)</div>
        <div><span style='background:#0000FF; width:20px; height:20px; display:inline-block;
            margin-right:10px; border:1px solid black;'></span>Linear Trend (3)</div>
    </div>
    """
)

from ipyleaflet import WidgetControl
exp4Map.add_control(WidgetControl(widget=legend_html_exp4, position='bottomleft'))

#  Show map
exp4Map


In [ ]:
# Step 11: Update exp_3class_grid to assign class 3 at those positions
exp4class = exp_3class_grid.where(linear_only_fill, 3)


In [ ]:
# Step 12: Assign class 5 where combined_mask == 1
exp5class = exp4class.where(combined_mask.eq(1), 4).rename('Exp5class')


In [ ]:
#  Reclassify exp5class: set to 6 where ecozone_mask == 0 (i.e., outside ecozone)
exp5class_export_ready = exp5class.where(ecozone_mask.eq(0), 6).rename('NDVI_ModelFit_5class')

# Clip to tile extent
exp5class_clipped = exp5class_export_ready.clip(tiles_fc.geometry())


In [ ]:
#  Rename both source images to avoid conflicts
exp_class_final_named = exp_class_final.rename('Original_Exp_Class')
exp5class_named = exp5class.rename('Exp5_Class_Source')

# Create a mask where exp5class is 1 or 2
exp5_mask = exp5class_named.eq(1).Or(exp5class_named.eq(2))

# Apply the mask: keep exp_class_final values only where exp5class is 1 or 2
exp_class_filtered = exp_class_final_named.where(exp5_mask.Not(), 0).rename('Filtered_Exp_Class_By_Exp5')

#  Visualization (optional, to confirm before export)
vis_params = {
    'min': 0,
    'max': 5,
    'palette': ['#D3D3D3', '#FF0000', '#FFA500', '#0000FF', '#00FFFF', '#008000']
}
Map = geemap.Map(center=[60, -96], zoom=3)
Map.addLayer(exp_class_filtered.clip(full_ecozone_fc), vis_params, ' Filtered Exp Class by Exp5 (New)')
Map


In [ ]:
#exporting exp trend class

In [ ]:
# Load tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()  # Convert to client-side integer

# Export settings
no_data_value = 9999
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

# Loop through tiles and create export tasks
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1  # Sequential numbering from 1

    description = f'exp5class_{tile_id}'
    file_name_prefix = f'exp5class_{tile_id}'

    image_to_export = exp5class_clipped.unmask(no_data_value).toInt16().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='linear_exp_combined',  # Folder name for this export
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print("All export tasks submitted. Go to the Earth Engine Code Editor 'Tasks' tab to run them manually.")


In [ ]:
#exporting trend slope grid values

In [ ]:
# Select the slope layer and rename
exp_slope_raw = per_pixel_exp_stack.select('Exp_Slope_Transformed')

# Set slope to 0 where exp5class is 3 or 5
exp_slope_masked_by_class = exp_slope_raw.where(
    exp5class.eq(3).Or(exp5class.eq(5)), 0
)

# Set slope to 0 where combined_mask == 1 (e.g., invalid or water/built-up)
exp_slope_masked_by_combined = exp_slope_masked_by_class.where(
    combined_mask.eq(1), 0
)

# Set slope to 0 outside the tile extent
tiles_extent = tiles_fc.geometry()
exp_slope_clipped = exp_slope_masked_by_combined.clip(tiles_extent)

# Optional: Round to 4 decimals
exp_slope_rounded = exp_slope_clipped.multiply(10000).round().divide(10000)


In [ ]:
# Load tiles again
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()  # Convert to client-side integer

# Export settings
no_data_value = 0  # We already zeroed out masked regions
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

#  Loop through tiles and export
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1

    description = f'expslope_{tile_id}'
    file_name_prefix = f'expslope_{tile_id}'

    image_to_export = exp_slope_rounded.clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='EXP_SLOPE',
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f"Export task submitted for tile {tile_id}: {description}")

print(" All export tasks submitted. Check the Earth Engine Code Editor 'Tasks' tab to monitor progress.")


In [ ]:
#  Load final classified image (assumed already computed)
#  Load tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()  # Convert to client-side integer

#  Export settings
no_data_value = 9999
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

#  Loop through tiles and create export tasks
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1  # Sequential numbering from 1

    description = f'linearclass_{tile_id}'
    file_name_prefix = f'linearclass_{tile_id}'

    image_to_export = final_class2_clipped.unmask(no_data_value).toInt16().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='LINEAR_TREND',  # Folder name updated
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print("All export tasks submitted. Go to the Earth Engine Code Editor 'Tasks' tab to run them manually.")


In [ ]:
#  Load tiles
tiles_fc = ee.FeatureCollection("projects/danielsiglab/assets/tileroi50")
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()

#  Export settings
no_data_value = 0  # Suitable for float rasters
crs = 'EPSG:4326'
crs_transform = [0.0002695, 0, -180, 0, -0.0002695, 90]

#  Loop through tiles and export slope
for i in range(num_tiles):
    tile = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id = i + 1

    description = f'linearslope_{tile_id}'
    file_name_prefix = f'linearslope_{tile_id}'

    image_to_export = final_static_slope_clipped.unmask(no_data_value).toFloat().clip(tile_geom)

    task = ee.batch.Export.image.toDrive(
        image=image_to_export,
        description=description,
        folder='linear_slope',  # Folder name for slope
        fileNamePrefix=file_name_prefix,
        region=tile_geom,
        crs=crs,
        crsTransform=crs_transform,
        maxPixels=1e13
    )

    task.start()
    print(f" Export task submitted for tile {tile_id}: {description}")

print(" All slope export tasks submitted. Check the Earth Engine 'Tasks' tab to monitor.")


In [ ]:
#From step 1-28 below is for mapping landcover type transitions into vegetation gain (greening),
#vegetation loss (browning), stable (no chnage),lateral vegatation shift, lateral non vegetation shift
#The code demontrate how mapped landcover transtion with VLCE2 (Lancover gridded dataset called "COMBINEDVCGL")
#The final export is 30m landcover coded grid that then enabled us to intersect NDVI trend class and landcover transiiton classes
#The Implementation was fully in Google Earth Engine Platform.

In [ ]:
// 1. Load VLCE2 land cover
var ca_lc = ee.ImageCollection("projects/sat-io/open-datasets/CA_FOREST_LC_VLCE2");

// 2. Load ecozones and exclude ECOZONE_ID 1 and 3
var ecozones = ee.FeatureCollection("projects/danielsiglab/assets/studyarea50");
var filteredEcozones = ecozones
  .filter(ee.Filter.neq('ID', 1))
  .filter(ee.Filter.neq('ID', 3));
var region = filteredEcozones.geometry();

// 3. Define comparison years
var compareYears = ee.List.sequence(1984, 2022).getInfo();

// 4. Define VLCE2 original classes and remapped 0–12
var originalClasses = [0, 20, 31, 32, 33, 40, 50, 80, 81, 100, 210, 220, 230];
var newSequentialClasses = ee.List.sequence(0, 12);

// 5. Function to reclassify land cover
function getReclassLC(year) {
  var raw = ca_lc
    .filter(ee.Filter.calendarRange(year, year, 'year'))
    .first()
    .select('b1');
  return raw.remap(originalClasses, newSequentialClasses).rename('LC');
}

// 6. Create baseline (mode of 1984–1986)
var baseline = ee.ImageCollection.fromImages([
  getReclassLC(1984),
  getReclassLC(1985),
  getReclassLC(1986)
]).reduce(ee.Reducer.mode()).rename('Baseline');

// 7. Define stable transitions: 0→0, ..., 12→12 = 1212
var noChangeCodes = ee.List.sequence(0, 12).map(function(x) {
  return ee.Number(x).multiply(100).add(x);
});

// 8. Track transitions and compute change metrics
var changeCount = ee.Image(0).rename('Change_Count');
var transitionImages = [];
var changedClassImages = [];

compareYears.forEach(function(year) {
  var currentLC = getReclassLC(year);
  var transition = baseline.multiply(100).add(currentLC).rename('Transition_' + year);
  transitionImages.push(transition);

  // Visualize transition
  Map.addLayer(transition.clip(region), {
    min: 0,
    max: 1212,
    palette: ['#cccccc', '#a6cee3', '#1f78b4', '#b2df8a', '#33a02c',
              '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6',
              '#6a3d9a', '#ffff99', '#b15928']
  }, 'Transition_' + year);

  // Count change years
  var changed = currentLC.neq(baseline);
  changeCount = changeCount.add(changed);

  // Mask stable pixels and rename
  var changedClass = currentLC.updateMask(changed).rename('LC_' + year);
  changedClassImages.push(changedClass);
});

// 9. Stack changed classes into a multiband image
var changedClassStack = ee.ImageCollection(changedClassImages).toBands();

// 10. Count number of unique changed classes
var uniqueChangeCount = changedClassStack
  .reduce(ee.Reducer.countDistinctNonNull())
  .rename('Unique_Class_Changes');

// 11. Unmask for completeness (important for full rendering!)
var changeCountFixed = changeCount.unmask(0).rename('Change_Count');
var uniqueChangeCountFixed = uniqueChangeCount.unmask(0).rename('Unique_Class_Changes');

// 12. Add map layers
Map.addLayer(baseline.clip(region), {
  min: 0, max: 12,
  palette: [
    '#686868', '#3333ff', '#ccffff', '#cccccc', '#996633', '#ffccff',
    '#ffff00', '#993399', '#9933cc', '#ccff33', '#006600', '#00cc00', '#cc9900'
  ]
}, 'Baseline (1984–1986)');

Map.addLayer(changeCountFixed.clip(region), {
  min: 0, max: 40,
  palette: ['#ffffff', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}, 'Total Change Years (Times Changed)');

Map.addLayer(uniqueChangeCountFixed.clip(region), {
 min: 0, max: 5,
 palette: ['#ffffff', '#d9f0a3', '#addd8e', '#78c679', '#41ab5d', '#238443', '#005a32']
}, 'Unique Changed Class Count');

//Map.addLayer(uniqueChangeCountFixed.clip(region), {
 // min: 0, max: 30,
 // palette: [
 //   '#440154', '#482475', '#414487', '#355f8d', '#2a788e',
 //   '#21908d', '#22a884', '#43bf71', '#7ad151', '#bddf26',
 //   '#fde725'
//  ]
//}, 'Unique Changed Class Count (Improved)');

// 13. Export results
//Export.image.toAsset({
//  image: changeCountFixed,
//  description: 'Export_Change_Count_Times_wo_eco1_3',
  //assetId: 'projects/danielsiglab/assets/change_count_times_1984_2022_wo_eco1_3',
  //region: region,
  //scale: 30,
  //maxPixels: 1e13
//});

//Export.image.toAsset({
  //image: uniqueChangeCountFixed,
  //description: 'Export_Unique_Class_Change_Count_Fixed_wo_eco1_3',
  //assetId: 'projects/danielsiglab/assets/unique_class_change_count_fixed_wo_eco1_3',
  //region: region,
  //scale: 30,
  //maxPixels: 1e13
//});

// 14. Center the map
Map.centerObject(region, 4);


// Step 15: Encode only years of actual landcover transitions from previous year
var encodedChangeYears = ee.Image(0).rename('Encoded_Years');
var lastLC = baseline;

// Loop over each year and encode only when the LC changes from the last year's LC
compareYears.forEach(function(year, index) {
  var currentLC = getReclassLC(year);
  var isChanged = currentLC.neq(lastLC);  // compare with previous year

  // Convert index to padded 2-digit position (e.g., '01' for 1984, '39' for 2022)
  var yearPos = ee.Number(index).add(1);
  var paddedCode = ee.Number.parse(yearPos.format('%02d'));

  // Only encode pixels that changed compared to the last year
  encodedChangeYears = encodedChangeYears.multiply(100).add(
    ee.Image.constant(paddedCode).updateMask(isChanged)
  ).unmask(encodedChangeYears);

  // Update lastLC for next iteration
  lastLC = currentLC;
});

// Rename for clarity
encodedChangeYears = encodedChangeYears.rename('Encoded_Years');

// 16. Visualize Encoded Change Years
Map.addLayer(encodedChangeYears.clip(region), {
  min: 0,
  max: 999999999999,  // Up to 6 transitions = 12 digits
  palette: ['white', 'black']
}, 'Encoded Change Years (2-digit position)');

// 17. Export Encoded Change Years
Export.image.toAsset({
  image: encodedChangeYears,
  description: 'Export_Encoded_Change_Years_wo_eco1_3',
  assetId: 'projects/danielsiglab/assets/encoded_change_years_wo_eco1_3',
  region: region,
  scale: 30,
  maxPixels: 1e13
});


//  Transition Layer Subsetting and Implementation
// 18. Convert transitionImages array to a properly structured ImageCollection

var transitionCollection = ee.ImageCollection.fromImages(
  transitionImages.map(function(img, index) {
    return ee.Image(img).set('year', compareYears[index]);
  })
);


//  19. Extract Transition Layers for 2020, 2021, 2022
var transition2020 = transitionCollection.filter(ee.Filter.eq('year', 2020)).first();
var transition2021 = transitionCollection.filter(ee.Filter.eq('year', 2021)).first();
var transition2022 = transitionCollection.filter(ee.Filter.eq('year', 2022)).first();

//  20. Define Class Codes for Greening, Browning, Lateral Shifts
//var greeningCodes = ee.List([5, 6, 7, 8, 9, 10, 11, 12, 105, 106, 107, 108, 109, 110, 111, 112,
 // 205, 206, 207, 208, 209, 210, 211, 212, 305, 306, 307, 308, 309, 310, 311, 312,
 // 405, 406, 407, 408, 409, 410, 411, 412, 510, 511, 512, 610, 611, 612, 910, 911, 912]);

//var browningCodes = ee.List([500, 501, 502, 503, 504, 600, 601, 602, 603, 604,
  //700, 701, 702, 703, 704, 800, 801, 802, 803, 804,
  //900, 901, 902, 903, 904, 1000, 1001, 1002, 1003,
  //1004, 1005, 1006, 1009, 1100, 1101, 1102, 1103,
 // 1104, 1105, 1106, 1109, 1200, 1201, 1202, 1203,
//  1204, 1205, 1206, 1209]);

//var lateralShiftCodes = ee.List([405, 406, 407, 408, 409, 410, 411, 412,
//504, 506, 507, 508, 509, 605, 607, 608, 609, 705, 706, 708, 709, 710, 711,
//712, 805, 806, 807, 809, 810, 811, 812, 905, 906, 907, 908, 1007, 1008, 1011,
//1012, 1107, 1108, 1110, 1112, 1207, 1208, 1210, 1211]);

//var lateralNonVegShiftCodes = ee.List([1, 2, 3, 4, 100, 102, 103, 104, 200, 201, 203, 204, 300, 301, 302, 304, 400, 401, 402, 403]);

//var stableCodes = ee.List([0, 101, 202, 303, 404, 505, 606, 707, 808, 909, 1010, 1111, 1212]);


// 20. Define Class Codes for Greening, Browning, Lateral Shifts
var greeningCodes = ee.List([5, 6, 7, 8, 9, 10, 11, 12, 105, 106, 107, 108, 109, 110, 111, 112,
    205, 206, 207, 208, 209, 210, 211, 212, 305, 306, 307, 308, 309, 310, 311, 312,
    405, 406, 407, 408, 409, 410, 411, 412, 506, 507, 508, 509, 510, 511, 512, 607, 610, 611, 612, 907, 910, 911, 912,1008]);

var browningCodes = ee.List([500, 501, 502, 503, 504, 600, 601, 602, 603, 604, 605,
    700, 701, 702, 703, 704, 800, 801, 802, 803, 804,806,
    900, 901, 902, 903, 904, 905, 1000, 1001, 1002, 1003,
    1004, 1005, 1006, 1009, 1100, 1101, 1102, 1103,
    1104, 1105, 1106, 1109, 1200, 1201, 1202, 1203,
    1204, 1205, 1206, 1209]);

var lateralShiftCodes = ee.List([405, 406, 407, 408, 409, 410, 411, 412, 506, 508, 509, 608,
609, 705, 706, 708, 709, 710, 711, 712, 805, 807, 809, 810, 811, 812, 906, 908, 1007, 1011,
1012, 1107, 1108, 1110, 1112, 1207, 1208, 1210, 1211]);

var lateralNonVegShiftCodes = ee.List([1, 2, 3, 4, 100, 102, 103, 104, 200, 201, 203, 204, 300, 301, 302, 304, 400, 401, 402, 403]);

var stableCodes = ee.List([0, 101, 202, 303, 404, 505, 606, 707, 808, 909, 1010, 1111, 1212]);



// 21. Function to Check Membership in a Code List
function isInList(img, codeList) {
  var masks = codeList.map(function(code) {
    var codeImage = ee.Image.constant(code);
    return img.eq(codeImage);
  });
  return ee.ImageCollection.fromImages(masks).max();
}

print('isInList function defined.');

// 22. Apply Mapping to Transition Collection
var green2020 = isInList(transition2020, greeningCodes);
var green2021 = isInList(transition2021, greeningCodes);
var green2022 = isInList(transition2022, greeningCodes);

var brown2020 = isInList(transition2020, browningCodes);
var brown2021 = isInList(transition2021, browningCodes);
var brown2022 = isInList(transition2022, browningCodes);

var lateral2020 = isInList(transition2020, lateralShiftCodes);
var lateral2021 = isInList(transition2021, lateralShiftCodes);
var lateral2022 = isInList(transition2022, lateralShiftCodes);



// 23. Determine the Final Classification
// Apply mapping for lateralNonVegShiftCodes
var nonVeg2020 = isInList(transition2020, lateralNonVegShiftCodes);
var nonVeg2021 = isInList(transition2021, lateralNonVegShiftCodes);
var nonVeg2022 = isInList(transition2022, lateralNonVegShiftCodes);


// 23–25. Two-year consistency only (no single-year fallback)
var initialClass = ee.Image(0)
  // Pass 1: must agree in 2022 & 2021
  .where(green2022.eq(1).and(green2021.eq(1)),     1)
  .where(brown2022.eq(1).and(brown2021.eq(1)),     2)
  .where(lateral2022.eq(1).and(lateral2021.eq(1)), 3)
  .where(nonVeg2022.eq(1).and(nonVeg2021.eq(1)),   4);

var finalClass = initialClass
  // Pass 2: for any still-zero pixel, must agree in 2022 & 2020
  .where(initialClass.eq(0)
         .and(green2022.eq(1).and(green2020.eq(1))),     1)
  .where(initialClass.eq(0)
         .and(brown2022.eq(1).and(brown2020.eq(1))),     2)
  .where(initialClass.eq(0)
         .and(lateral2022.eq(1).and(lateral2020.eq(1))), 3)
  .where(initialClass.eq(0)
         .and(nonVeg2022.eq(1).and(nonVeg2020.eq(1))),   4);

// Everything else remains 0 (stable)


// 25. Set Stable Class
finalClass = finalClass.where(
  finalClass.eq(0), 0
);

// 26. Visualization and Export
Map.addLayer(finalClass.clip(region), {
  min: 0, max: 3,
  palette: ['#0000FF', '#00FF00', '#FF0000', '#00FFFF', '#FFA500']
}, 'Ecological Change Classification');

//Export.image.toAsset({
 // image: finalClass,
 // description: 'Ecological_Change_Class_2020_2022_Majority_2022_Focus',
 // assetId: 'projects/danielsiglab/assets/ecological_change_class_2020_2022_majority_2022_focus',
 // region: region,
  //scale: 30,
  //maxPixels: 1e13
//});

print('Ecological Change Classification with 2022 Focus and Majority Logic completed.');


// 27. Patch 2022 VLCE2 transition with stable codes where finalClass == 0
// Compute Baseline→Baseline “no-change” code
var stableVLCE = baseline.multiply(100).add(baseline);

// Pull the original 2022 transition
var rawT2022 = transitionCollection
  .filter(ee.Filter.eq('year', 2022))
  .first();

// Create a patched transition:
// where finalClass ≠ 0, keep rawT2022;
//  where finalClass == 0, write stableVLCE instead.
var transition2022_Patched = rawT2022.where(
  finalClass.eq(0),
  stableVLCE
).rename('Transition_2022_Patched');

// Optional: visualize to check
Map.addLayer(transition2022_Patched.clip(region), {
  min: 0, max: 1212,
  palette: [
    '#ffffff','#cccccc','#a6cee3','#1f78b4','#b2df8a','#33a02c',
    '#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a',
    '#ffff99','#b15928'
  ]
}, 'Patched Transition 2022 VLCE2');

//28 Export of landcover harmonized transition grid

//  Load your tile grid and your finalClass image
//var tilesFC    = ee.FeatureCollection("projects/danielsiglab/assets/tilenoneco3_74");
//var finalClass = finalClass;  // Make sure you've defined finalClass earlier in your script

//  Define NoData, CRS & transform (matching your example)
//var noDataVal    = 999;
//var crs          = 'EPSG:4326';
//var crsTransform = [0.0002695, 0, -180, 0, -0.0002695, 90];

//  Convert tile collection to a client-side list
//var tileList = tilesFC.toList(tilesFC.size());
//var numTiles = tileList.size().getInfo();

//  Loop over tiles and export finalClass clipped to each
//for (var i = 0; i < numTiles; i++) {
 // var tile   = ee.Feature(tileList.get(i));
  //var geom   = tile.geometry();
  //var tileID = i + 1;  // sequential numbering

  //Export.image.toDrive({
    //image:          finalClass
    //                 .unmask(noDataVal)
    //                  .toInt16()   // class codes are integer
    //                  .clip(geom),
   // description:     'FinalClass_Tile_' + tileID,
   // folder:          'CORRECTVCLE2_ECO_CHANGE',   // ← Your specified folder
    //fileNamePrefix:  'final_class_tile_' + tileID,
   // crs:             crs,
   // crsTransform:    crsTransform,
    //region:          geom,
   // maxPixels:       1e13
  //});

 // print('Submitted export for FinalClass tile', tileID);
//}

//print(' All FinalClass export tasks submitted.');


In [ ]:
#Estimating the Energy Trend from ERA5-LAND REANALYSIS

In [ ]:
#  Define study area
studyarea50 = full_ecozone_fc

#  Define years
start_year = 1986
end_year   = 2023
year_list  = ee.List.sequence(start_year, end_year)

# Define season months (change as needed)
summer_months = ee.List([3, 4, 5])  # e.g., Spring = Mar–May; set [6,7,8,9] for Jun–Sep

# Load ERA5-Land monthly & add 'month'
era5land = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY')
def add_month(img):
    date = ee.Date(img.get('system:time_start'))
    return img.set('month', date.get('month'))
era5land_with_month = era5land.map(add_month)

#  Seasonal image with H, LE, Rns, Rnl, forecast_albedo, imbalance, and Bowen ratio (H/LE)
def get_season_energy_image(y):
    y = ee.Number(y)

    season_ic = (era5land_with_month
        .filterDate(ee.Date.fromYMD(y, 1, 1), ee.Date.fromYMD(y, 12, 31))
        .filter(ee.Filter.inList('month', summer_months))        #  use your months list
        .filterBounds(studyarea50.geometry()))                    #  robust clip target

    return ee.Algorithms.If(
        season_ic.size().eq(0),
        None,
        (
            # Mean seasonal composites
            season_ic.select([
                'surface_sensible_heat_flux',
                'surface_latent_heat_flux',
                'surface_net_solar_radiation',
                'surface_net_thermal_radiation',
                'forecast_albedo'
            ]).mean().rename([
                'sensible_flux',    # H
                'latent_flux',      # LE
                'net_solar_rad',    # Rns
                'net_thermal_rad',  # Rnl
                'forecast_albedo'   # ERA5-Land albedo band
            ])
            # Imbalance = Rns + Rnl − (LE + H)
            .addBands(
                season_ic.mean().expression(
                    'solar + thermal - (latent + sensible)',
                    {
                        'solar':    season_ic.select('surface_net_solar_radiation').mean(),
                        'thermal':  season_ic.select('surface_net_thermal_radiation').mean(),
                        'latent':   season_ic.select('surface_latent_heat_flux').mean(),
                        'sensible': season_ic.select('surface_sensible_heat_flux').mean()
                    }
                ).rename('imbalance')
            )
            # Bowen ratio = H / LE (mask tiny LE to avoid blow-ups)
            .addBands(
                ee.Image().expression(
                    'H / (LE + 1e-9)',
                    {
                        'H':  season_ic.select('surface_sensible_heat_flux').mean(),
                        'LE': season_ic.select('surface_latent_heat_flux').mean()
                    }
                ).updateMask(
                    season_ic.select('surface_latent_heat_flux').mean().gt(1e-9)
                ).rename('bowen_ratio')
            )
            .clip(studyarea50.geometry())
            .set({
                'year': y,
                'system:time_start': ee.Date.fromYMD(y, 7, 1).millis()  # mid-year stamp for time slider/debug
            })
        )
    )

#  Build list → ImageCollection
all_season_energy_images        = year_list.map(get_season_energy_image)
all_season_energy_images_clean  = all_season_energy_images.removeAll([None])
full_season_energy_collection   = ee.ImageCollection.fromImages(all_season_energy_images_clean)

print(' Seasonal ERA5-Land collection size:', full_season_energy_collection.size().getInfo())
print(' Bands sample:', full_season_energy_collection.first().bandNames().getInfo())


In [ ]:
#  Define tiling function for the ENERGY collection
def split_energy_to_tiles(tile_feature):
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id   = tile_feature.get('tile_id')  # safer than getString

    def clip_and_tag(img):
        return (ee.Image(img)
                .clip(tile_geom)
                .set('tile_id', tile_id)
                .set('year', img.get('year'))
                .set('system:time_start', img.get('system:time_start')))

    return full_season_energy_collection.map(clip_and_tag)

#  Apply to all tiles → returns List of ImageCollections
all_tiles_energy_images = tiles_fc.map(split_energy_to_tiles)

#  Flatten and wrap as ImageCollection
energy_tiles_collection = ee.ImageCollection(all_tiles_energy_images.flatten())

#  Quick sanity checks
print(' Energy tiles collection ready. Count:', energy_tiles_collection.size().getInfo())
first_img = energy_tiles_collection.first()
print('Bands in first tiled image:', first_img.bandNames().getInfo())
print('Sample props:', first_img.toDictionary(['tile_id','year']).getInfo())


In [ ]:
#  Define tiling function for the ENERGY collection
def split_energy_to_tiles(tile_feature):
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id = tile_feature.getString('tile_id')

    # Clip and tag each energy image for this tile
    def clip_and_tag(img):
        return ee.Image(img) \
            .clip(tile_geom) \
            .set('tile_id', tile_id) \
            .set('year', img.get('year')) \
            .set('system:time_start', img.get('system:time_start'))

    #  Return as ImageCollection (just like climate)
    return full_season_energy_collection.map(clip_and_tag)

# Apply to all tiles → returns List of ImageCollections
all_tiles_energy_images = tiles_fc.map(split_energy_to_tiles)

# Flatten these ImageCollections
flattened_energy_images = all_tiles_energy_images.flatten()

# Convert to ImageCollection
energy_tiles_collection = ee.ImageCollection(flattened_energy_images)

#  Print result
print('Energy tiles collection ready. Count:', energy_tiles_collection.size().getInfo())


In [ ]:
# ===========================
# PER-TILE TRENDS (H, LE, Imbalance, Albedo, Bowen) + PREVIEW MAP
# ===========================

# --- Trend per tile (uses energy_tiles_collection you just built) ---
def compute_energy_trend_tile(tile_feature):
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id   = tile_feature.get('tile_id')  # keep original type

    # this tile's time series
    tile_col = energy_tiles_collection.filter(ee.Filter.eq('tile_id', tile_id))
    years = ee.List(tile_col.aggregate_array('year')).distinct().sort()

    # build stack [H, LE, imbalance, albedo, bowen] + relativeYear
    def build_trend_image(y):
        y = ee.Number(y)
        img = ee.Image(tile_col.filter(ee.Filter.eq('year', y)).first())
        return ee.Algorithms.If(
            img,
            img.addBands(
                ee.Image.constant(y.subtract(start_year)).rename('relativeYear').toFloat()
            ).clip(tile_geom).set('year', y),
            None
        )

    series_ic = ee.ImageCollection.fromImages(years.map(build_trend_image)).filter(ee.Filter.notNull(['relativeYear']))

    # if too few years, return masked output with correct band schema
    out_bands = [
        'SenSlope_sensible','SenSlope_latent','SenSlope_imbalance','SenSlope_albedo','SenSlope_bowen',
        'Tau_sensible','Tau_latent','Tau_imbalance','Tau_albedo','Tau_bowen',
        'Z_sensible','Z_latent','Z_imbalance','Z_albedo','Z_bowen'
    ]
    empty = (ee.Image.constant([0]*len(out_bands)).rename(out_bands)
             .updateMask(ee.Image(0)).clip(tile_geom).set('tile_id', tile_id))

    cond = series_ic.size().gt(5)  # require ≥6 years

    def compute():
        # helpers
        def sen(var):
            return (series_ic.select(['relativeYear', var])
                    .reduce(ee.Reducer.sensSlope())
                    .select('slope'))
        def ktau(var):
            # kendallsCorrelation -> bands: 'tau', 'p-value'
            return (series_ic.select(['relativeYear', var])
                    .reduce(ee.Reducer.kendallsCorrelation())
                    .select('tau'))

        # slopes
        sen_sensible  = sen('sensible_flux').rename('SenSlope_sensible')
        sen_latent    = sen('latent_flux').rename('SenSlope_latent')
        sen_imbalance = sen('imbalance').rename('SenSlope_imbalance')
        sen_albedo    = sen('forecast_albedo').rename('SenSlope_albedo')
        sen_bowen     = sen('bowen_ratio').rename('SenSlope_bowen')

        # tau
        tau_sensible  = ktau('sensible_flux').rename('Tau_sensible')
        tau_latent    = ktau('latent_flux').rename('Tau_latent')
        tau_imbalance = ktau('imbalance').rename('Tau_imbalance')
        tau_albedo    = ktau('forecast_albedo').rename('Tau_albedo')
        tau_bowen     = ktau('bowen_ratio').rename('Tau_bowen')

        # Z from tau using actual N for this tile
        N = series_ic.size()
        def z_of(tau_img):
            return (tau_img.multiply(3)
                    .multiply(N.multiply(N.subtract(1)).sqrt())
                    .divide(ee.Number(2).multiply(N.multiply(2).add(5).sqrt())))
        z_sensible = z_of(tau_sensible).rename('Z_sensible')
        z_latent   = z_of(tau_latent).rename('Z_latent')
        z_imb      = z_of(tau_imbalance).rename('Z_imbalance')
        z_albedo   = z_of(tau_albedo).rename('Z_albedo')
        z_bowen    = z_of(tau_bowen).rename('Z_bowen')

        return (sen_sensible.addBands([
                    sen_latent, sen_imbalance, sen_albedo, sen_bowen,
                    tau_sensible, tau_latent, tau_imbalance, tau_albedo, tau_bowen,
                    z_sensible, z_latent, z_imb, z_albedo, z_bowen
                ])
                .set('tile_id', tile_id)
                .set('system:time_start', ee.Date.fromYMD(end_year, 7, 1).millis())
                .clip(tile_geom))

    return ee.Image(ee.Algorithms.If(cond, compute(), empty))

# run trends per tile
by_tile_energy_trends = ee.ImageCollection(tiles_fc.map(compute_energy_trend_tile))
print('Trend images per tile:', by_tile_energy_trends.size().getInfo())

# mosaic Sen slopes
sensible_sen_mosaic  = by_tile_energy_trends.select('SenSlope_sensible').mosaic()
latent_sen_mosaic    = by_tile_energy_trends.select('SenSlope_latent').mosaic()
imbalance_sen_mosaic = by_tile_energy_trends.select('SenSlope_imbalance').mosaic()
albedo_sen_mosaic    = by_tile_energy_trends.select('SenSlope_albedo').mosaic()
bowen_sen_mosaic     = by_tile_energy_trends.select('SenSlope_bowen').mosaic()

print('Slope mosaics ready')




In [ ]:
# Per-tile ENERGY Trend Fitting including IMBALANCE + ALBEDO + BOWEN
def compute_energy_trend_tile(tile_feature):
    tile_feature = ee.Feature(tile_feature)
    tile_geom = tile_feature.geometry()
    tile_id   = tile_feature.getString('tile_id')

    # Filter this tile's time series
    tile_col = energy_tiles_collection.filter(ee.Filter.eq('tile_id', tile_id))

    # Distinct, sorted years
    years = ee.List(tile_col.aggregate_array('year')).distinct().sort()

    # Build per-year stack with imbalance, relativeYear
    def build_trend_image(y):
        y = ee.Number(y)
        img = ee.Image(tile_col.filter(ee.Filter.eq('year', y)).first())
        return ee.Algorithms.If(
            img,
            (img.addBands(
                 # (Re)compute imbalance: Rns + Rnl − (LE + H)
                 img.expression(
                     'solar + thermal - (latent + sensible)',
                     {
                         'solar':    img.select('net_solar_rad'),
                         'thermal':  img.select('net_thermal_rad'),
                         'latent':   img.select('latent_flux'),
                         'sensible': img.select('sensible_flux')
                     }
                 ).rename('imbalance')
             )
             .addBands(ee.Image.constant(y.subtract(start_year)).rename('relativeYear').toFloat())
             .clip(tile_geom)
             .set('year', y)),
            None
        )

    series_ic = ee.ImageCollection.fromImages(years.map(build_trend_image)).filter(ee.Filter.notNull(['relativeYear']))

    # If too few years, return masked output with correct band schema
    out_bands = [
        'SenSlope_sensible','SenSlope_latent','SenSlope_imbalance','SenSlope_albedo','SenSlope_bowen',
        'Tau_sensible','Tau_latent','Tau_imbalance','Tau_albedo','Tau_bowen',
        'Z_sensible','Z_latent','Z_imbalance','Z_albedo','Z_bowen'
    ]
    empty = (ee.Image.constant([0]*len(out_bands)).rename(out_bands)
             .updateMask(ee.Image(0)).clip(tile_geom)
             .set('tile_id', tile_id))

    cond = series_ic.size().gt(5)  # require ≥6 years

    def compute():
        # Helpers
        def sen(var):
            return (series_ic.select(['relativeYear', var])
                    .reduce(ee.Reducer.sensSlope())
                    .select('slope'))

        def ktau(var):
            # kendallsCorrelation returns bands: 'tau', 'p-value'
            return (series_ic.select(['relativeYear', var])
                    .reduce(ee.Reducer.kendallsCorrelation())
                    .select('tau'))

        # Sen's slopes
        sen_sensible  = sen('sensible_flux').rename('SenSlope_sensible')
        sen_latent    = sen('latent_flux').rename('SenSlope_latent')
        sen_imbalance = sen('imbalance').rename('SenSlope_imbalance')
        sen_albedo    = sen('forecast_albedo').rename('SenSlope_albedo')
        sen_bowen     = sen('bowen_ratio').rename('SenSlope_bowen')

        # Kendall's tau
        tau_sensible  = ktau('sensible_flux').rename('Tau_sensible')
        tau_latent    = ktau('latent_flux').rename('Tau_latent')
        tau_imbalance = ktau('imbalance').rename('Tau_imbalance')
        tau_albedo    = ktau('forecast_albedo').rename('Tau_albedo')
        tau_bowen     = ktau('bowen_ratio').rename('Tau_bowen')

        # Z from tau (large-sample approx) using actual N for this tile
        N = series_ic.size()
        def z_of(tau_img):
            return (tau_img.multiply(3)
                    .multiply(N.multiply(N.subtract(1)).sqrt())
                    .divide(ee.Number(2).multiply(N.multiply(2).add(5).sqrt())))
        z_sensible = z_of(tau_sensible).rename('Z_sensible')
        z_latent   = z_of(tau_latent).rename('Z_latent')
        z_imb      = z_of(tau_imbalance).rename('Z_imbalance')
        z_albedo   = z_of(tau_albedo).rename('Z_albedo')
        z_bowen    = z_of(tau_bowen).rename('Z_bowen')

        return (sen_sensible.addBands([
                    sen_latent, sen_imbalance, sen_albedo, sen_bowen,
                    tau_sensible, tau_latent, tau_imbalance, tau_albedo, tau_bowen,
                    z_sensible, z_latent, z_imb, z_albedo, z_bowen
                ])
                .set('tile_id', tile_id)
                .set('system:time_start', ee.Date.fromYMD(end_year, 7, 1).millis())
                .clip(tile_geom))

    return ee.Image(ee.Algorithms.If(cond, compute(), empty))


In [ ]:
#Apply to all tiles
by_tile_energy_trends = ee.ImageCollection(
    tiles_fc.map(compute_energy_trend_tile)
)

print('Energy trend fitting per tile complete!')


In [ ]:

# Mosaic Sen's Slopes for all variables (H, LE, Imbalance, Albedo, Bowen)
sensible_sen_mosaic  = by_tile_energy_trends.select('SenSlope_sensible').mosaic()
latent_sen_mosaic    = by_tile_energy_trends.select('SenSlope_latent').mosaic()
imbalance_sen_mosaic = by_tile_energy_trends.select('SenSlope_imbalance').mosaic()
albedo_sen_mosaic    = by_tile_energy_trends.select('SenSlope_albedo').mosaic()
bowen_sen_mosaic     = by_tile_energy_trends.select('SenSlope_bowen').mosaic()

print('Energy slope mosaics ready!')

#  Visualize in geemap
Map2 = geemap.Map(center=[60, -100], zoom=4)

# Flux slopes (units depend on your seasonal aggregation; keep same scale as before)
flux_viz = {'min': -10, 'max': 10, 'palette': ['blue', 'white', 'red']}

# Albedo slope (unitless per year). Adjust range if too tight/loose.
albedo_viz = {'min': -0.03, 'max': 0.03, 'palette': ['blue', 'white', 'red']}

# Bowen ratio slope (per year). Adjust if needed.
bowen_viz = {'min': -0.2, 'max': 0.2, 'palette': ['blue', 'white', 'red']}

Map2.addLayer(sensible_sen_mosaic,  flux_viz,  'SenSlope Sensible Heat (H)')
Map2.addLayer(latent_sen_mosaic,    flux_viz,  'SenSlope Latent Heat (LE)')
Map2.addLayer(imbalance_sen_mosaic, flux_viz,  'SenSlope Imbalance (Rn−H−LE)')
Map2.addLayer(albedo_sen_mosaic,    albedo_viz,'SenSlope Albedo (forecast_albedo)')
Map2.addLayer(bowen_sen_mosaic,     bowen_viz, 'SenSlope Bowen Ratio (H/LE)')

Map2.addLayerControl()
Map2


In [ ]:
# ===========================
# SPRING — EXPORT (masked by Z)
# ===========================
z_threshold   = 0.84
noDataVal     = 999
era5_transform = [0.1, 0, -180, 0, -0.1, 90]
crs           = 'EPSG:4326'

# 1) Z mosaics
z_sensible_mosaic  = by_tile_energy_trends.select('Z_sensible').mosaic()
z_latent_mosaic    = by_tile_energy_trends.select('Z_latent').mosaic()
z_imbalance_mosaic = by_tile_energy_trends.select('Z_imbalance').mosaic()
z_albedo_mosaic    = by_tile_energy_trends.select('Z_albedo').mosaic()
z_bowen_mosaic     = by_tile_energy_trends.select('Z_bowen').mosaic()

print("Spring Z-score mosaics ready!")

# 2) Mask slopes with Z-threshold (keep Float32 to preserve small values)
masked_sensible_sen  = sensible_sen_mosaic.updateMask(z_sensible_mosaic.abs().gte(z_threshold)).unmask(noDataVal).toFloat()
masked_latent_sen    = latent_sen_mosaic.updateMask(z_latent_mosaic.abs().gte(z_threshold)).unmask(noDataVal).toFloat()
masked_imbalance_sen = imbalance_sen_mosaic.updateMask(z_imbalance_mosaic.abs().gte(z_threshold)).unmask(noDataVal).toFloat()
masked_albedo_sen    = albedo_sen_mosaic.updateMask(z_albedo_mosaic.abs().gte(z_threshold)).unmask(noDataVal).toFloat()
masked_bowen_sen     = bowen_sen_mosaic.updateMask(z_bowen_mosaic.abs().gte(z_threshold)).unmask(noDataVal).toFloat()

print('Spring masking and NoData assignment complete!')

# 3) Loop & export each tile to Drive (SPRING)
tile_list = tiles_fc.toList(tiles_fc.size())
num_tiles = tile_list.size().getInfo()
print(' Number of tiles to export (SPRING):', num_tiles)

for i in range(num_tiles):
    tile     = ee.Feature(tile_list.get(i))
    tile_geom = tile.geometry()
    tile_id   = i + 1
    region    = tile_geom.bounds().getInfo()['coordinates']

    # H (sensible)
    task_h = ee.batch.Export.image.toDrive(
        image=masked_sensible_sen.clip(tile_geom),
        description=f"SPRING_Tile_ERA5_Sensible_{tile_id}",
        folder='ERA5_SPRING_SENSIBLETREND',
        fileNamePrefix=f"SPRING_Tile_ERA5_Sensible_{tile_id}",
        crs=crs,
        crsTransform=era5_transform,
        region=region,
        maxPixels=1e13
    ); task_h.start()

    # LE (latent)
    task_le = ee.batch.Export.image.toDrive(
        image=masked_latent_sen.clip(tile_geom),
        description=f"SPRING_Tile_ERA5_Latent_{tile_id}",
        folder='ERA5_SPRING_LATENTTREND',
        fileNamePrefix=f"SPRING_Tile_ERA5_Latent_{tile_id}",
        crs=crs,
        crsTransform=era5_transform,
        region=region,
        maxPixels=1e13
    ); task_le.start()

    # Imbalance
    task_imb = ee.batch.Export.image.toDrive(
        image=masked_imbalance_sen.clip(tile_geom),
        description=f"SPRING_Tile_ERA5_Imbalance_{tile_id}",
        folder='ERA5_SPRING_IMBALANCETREND',
        fileNamePrefix=f"SPRING_Tile_ERA5_Imbalance_{tile_id}",
        crs=crs,
        crsTransform=era5_transform,
        region=region,
        maxPixels=1e13
    ); task_imb.start()

    # Albedo (forecast_albedo)
    task_alb = ee.batch.Export.image.toDrive(
        image=masked_albedo_sen.clip(tile_geom),
        description=f"SPRING_Tile_ERA5_Albedo_{tile_id}",
        folder='ERA5_SPRING_ALBEDOTREND',
        fileNamePrefix=f"SPRING_Tile_ERA5_Albedo_{tile_id}",
        crs=crs,
        crsTransform=era5_transform,
        region=region,
        maxPixels=1e13
    ); task_alb.start()

    # Bowen ratio (H/LE)
    task_br = ee.batch.Export.image.toDrive(
        image=masked_bowen_sen.clip(tile_geom),
        description=f"SPRING_Tile_ERA5_Bowen_{tile_id}",
        folder='ERA5_SPRING_BOWENTREND',
        fileNamePrefix=f"SPRING_Tile_ERA5_Bowen_{tile_id}",
        crs=crs,
        crsTransform=era5_transform,
        region=region,
        maxPixels=1e13
    ); task_br.start()

    print(f" Spring export tasks started for tile {tile_id}")

print('All Spring ERA5-Land export tasks submitted!')


In [ ]:
#Energy Flux Trend Computations and decomposition

In [ ]:
import arcpy, os
from arcpy.sa import Raster, ExtractByMask, SetNull, Abs

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

gdb = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
arcpy.env.workspace = gdb

# Mask to use (INSIDE area of this dataset is kept).
# Works if it's a raster or a polygon feature class.
mask_ds = os.path.join(gdb, "StudyArea_by1deglat_ras")

# Keep alignment consistent
arcpy.env.snapRaster = mask_ds

# Only these three rasters, taken directly by their exact names from the GDB
targets = ["spring_sens_wm2", "spring_latent_wm2", "spring_imb_wm2"]

# Value to convert to NoData (with small tolerance for float equality)
val = 0.003858
tol = 1e-06

for name in targets:
    in_raster = Raster(os.path.join(gdb, name))

    # 1) Extract by Mask (INSIDE area of mask_ds)
    masked = ExtractByMask(in_raster, mask_ds)
    tmp = os.path.join("in_memory", f"{name}_masked")
    masked.save(tmp)

    # 2) Set cells equal to 0.003858 to NoData
    cleaned = SetNull(Abs(Raster(tmp) - val) <= tol, Raster(tmp))
    out = os.path.join(gdb, f"{name}_clean")
    cleaned.save(out)

    print(f"Done: {name} → {os.path.basename(out)}")

print("Finished all three rasters.")


In [ ]:
import arcpy, os, math
from arcpy.sa import ZonalStatisticsAsTable

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

# --- Paths ---
gdb = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
arcpy.env.workspace = gdb

# Feature class with zones
latbands_fc = r"U:\\class_transition\landcover_transition\landcover_transition.gdb\Study_tse"

# Choose your zone field: "Lat_band" or "Value"
zone_field = "Name"   # <-- using 'Value' as requested

# Input rasters
spring_rasters = [
    "fal_imb_wm2_clean",
    "fal_sens_wm2_clean",
    "fal_latent_wm2_clean"
]

# Env: use native raster cell size
arcpy.env.cellSize = 0.1

# --- Constants for pixel area calculation ---
R = 6371008.7714  # meters
deg2rad = math.pi / 180.0
dlat_deg = 0.1
dlon_deg = 0.1
dlon_rad = dlon_deg * deg2rad

def mid_lat_from_band(band_text):
    """ Parse '44–45N' -> 44.5. """
    s = band_text.replace("–", "-").replace("N", "").strip()
    a, b = s.split("-")
    return (float(a) + float(b)) / 2.0

def pixel_area_m2_at_lat(lat_deg):
    """ Exact spherical formula for a 0.1° x 0.1° cell at given latitude. """
    phi1 = (lat_deg - dlat_deg/2.0) * deg2rad
    phi2 = (lat_deg + dlat_deg/2.0) * deg2rad
    return (R**2) * dlon_rad * (math.sin(phi2) - math.sin(phi1))

# --- Precompute centroid latitudes for zones (for any non-Lat_band field) ---
ecozone_lat = {}
if zone_field != "Lat_band":
    with arcpy.da.SearchCursor(latbands_fc, [zone_field, "SHAPE@XY"]) as sc:
        for zid, (x, y) in sc:
            ecozone_lat[zid] = y  # centroid latitude in degrees

# Where to write Excel / CSV (fixed raw string)
export_dir = r"X:\\"

# --- Main loop ---
for rname in spring_rasters:
    in_ras = os.path.join(gdb, rname)

    # Distinguish output table name
    if zone_field == "Lat_band":
        out_tbl = os.path.join(gdb, f"{rname}_zstats_latbands")
    else:
        out_tbl = os.path.join(gdb, f"eco_{rname}_zstats")

    # --- Run Zonal Statistics ---
    ZonalStatisticsAsTable(
        in_zone_data=latbands_fc,
        zone_field=zone_field,
        in_value_raster=in_ras,
        out_table=out_tbl,
        ignore_nodata="DATA",
        statistics_type="ALL"
    )

    # --- Add area fields ---
    add_fields = [
        ("LAT_MID", "DOUBLE"),
        ("PIX_AREA_M2", "DOUBLE"),
        ("VALID_AREA_M2", "DOUBLE"),
        ("VALID_AREA_KM2", "DOUBLE")
    ]
    existing = {f.name for f in arcpy.ListFields(out_tbl)}
    for fname, ftype in add_fields:
        if fname not in existing:
            arcpy.management.AddField(out_tbl, fname, ftype)

    # --- Update areas ---
    with arcpy.da.UpdateCursor(
        out_tbl, [zone_field, "COUNT", "LAT_MID", "PIX_AREA_M2", "VALID_AREA_M2", "VALID_AREA_KM2"]
    ) as cur:
        for zname, cnt, lat_mid, pix_m2, area_m2, area_km2 in cur:
            if zone_field == "Lat_band":
                lat_mid = mid_lat_from_band(zname)
            else:
                lat_mid = ecozone_lat.get(zname, 0.0)

            pix_m2  = pixel_area_m2_at_lat(lat_mid)
            area_m2 = (cnt or 0) * pix_m2
            area_km2 = area_m2 / 1e6
            cur.updateRow((zname, cnt, lat_mid, pix_m2, area_m2, area_km2))

    # --- Exports ---
    base = os.path.splitext(os.path.basename(out_tbl))[0]

    # Excel export
    xlsx_path = os.path.join(export_dir, f"{base}.xlsx")
    arcpy.conversion.TableToExcel(
        Input_Table=out_tbl,
        Output_Excel_File=xlsx_path,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )

    # CSV export (optional — keep if you want .csv as well)
    csv_path = os.path.join(export_dir, f"{base}.csv")
    arcpy.conversion.TableToTable(
        in_rows=out_tbl,
        out_path=export_dir,
        out_name=f"{base}.csv"
    )

    print(f" Done: {os.path.basename(out_tbl)}  |  XLSX: {xlsx_path}  |  CSV: {csv_path}")

print(" All zonal tables created with VALID_AREA_M2/KM2 based on 0.1° pixels and exported to Excel/CSV.")


In [ ]:
#using shapefile for the ecozones

In [ ]:
import arcpy, os, math
from arcpy.sa import ZonalStatisticsAsTable

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

# --- Paths ---
gdb = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
arcpy.env.workspace = gdb

# Feature class with zones
latbands_fc = r"U:\\class_transition\landcover_transition\landcover_transition.gdb\Study_tse"

# Choose your zone field: "Lat_band" or "Value"
zone_field = "Name"   # <-- using 'Value' as requested

# Input rasters
spring_rasters = [
    "fal_imb_wm2_clean",
    "fal_sens_wm2_clean",
    "fal_latent_wm2_clean"
]

# Env: use native raster cell size
arcpy.env.cellSize = 0.1

# --- Constants for pixel area calculation ---
R = 6371008.7714  # meters
deg2rad = math.pi / 180.0
dlat_deg = 0.1
dlon_deg = 0.1
dlon_rad = dlon_deg * deg2rad

def mid_lat_from_band(band_text):
    """ Parse '44–45N' -> 44.5. """
    s = band_text.replace("–", "-").replace("N", "").strip()
    a, b = s.split("-")
    return (float(a) + float(b)) / 2.0

def pixel_area_m2_at_lat(lat_deg):
    """ Exact spherical formula for a 0.1° x 0.1° cell at given latitude. """
    phi1 = (lat_deg - dlat_deg/2.0) * deg2rad
    phi2 = (lat_deg + dlat_deg/2.0) * deg2rad
    return (R**2) * dlon_rad * (math.sin(phi2) - math.sin(phi1))

# --- Precompute centroid latitudes for zones (for any non-Lat_band field) ---
ecozone_lat = {}
if zone_field != "Lat_band":
    with arcpy.da.SearchCursor(latbands_fc, [zone_field, "SHAPE@XY"]) as sc:
        for zid, (x, y) in sc:
            ecozone_lat[zid] = y  # centroid latitude in degrees

# Where to write Excel / CSV (fixed raw string)
export_dir = r"X:\\"

# --- Main loop ---
for rname in spring_rasters:
    in_ras = os.path.join(gdb, rname)

    # Distinguish output table name
    if zone_field == "Lat_band":
        out_tbl = os.path.join(gdb, f"{rname}_zstats_latbands")
    else:
        out_tbl = os.path.join(gdb, f"eco_{rname}_zstats")

    # --- Run Zonal Statistics ---
    ZonalStatisticsAsTable(
        in_zone_data=latbands_fc,
        zone_field=zone_field,
        in_value_raster=in_ras,
        out_table=out_tbl,
        ignore_nodata="DATA",
        statistics_type="ALL"
    )

    # --- Add area fields ---
    add_fields = [
        ("LAT_MID", "DOUBLE"),
        ("PIX_AREA_M2", "DOUBLE"),
        ("VALID_AREA_M2", "DOUBLE"),
        ("VALID_AREA_KM2", "DOUBLE")
    ]
    existing = {f.name for f in arcpy.ListFields(out_tbl)}
    for fname, ftype in add_fields:
        if fname not in existing:
            arcpy.management.AddField(out_tbl, fname, ftype)

    # --- Update areas ---
    with arcpy.da.UpdateCursor(
        out_tbl, [zone_field, "COUNT", "LAT_MID", "PIX_AREA_M2", "VALID_AREA_M2", "VALID_AREA_KM2"]
    ) as cur:
        for zname, cnt, lat_mid, pix_m2, area_m2, area_km2 in cur:
            if zone_field == "Lat_band":
                lat_mid = mid_lat_from_band(zname)
            else:
                lat_mid = ecozone_lat.get(zname, 0.0)

            pix_m2  = pixel_area_m2_at_lat(lat_mid)
            area_m2 = (cnt or 0) * pix_m2
            area_km2 = area_m2 / 1e6
            cur.updateRow((zname, cnt, lat_mid, pix_m2, area_m2, area_km2))

    # --- Exports ---
    base = os.path.splitext(os.path.basename(out_tbl))[0]

    # Excel export
    xlsx_path = os.path.join(export_dir, f"{base}.xlsx")
    arcpy.conversion.TableToExcel(
        Input_Table=out_tbl,
        Output_Excel_File=xlsx_path,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )

    # CSV export (optional — keep if you want .csv as well)
    csv_path = os.path.join(export_dir, f"{base}.csv")
    arcpy.conversion.TableToTable(
        in_rows=out_tbl,
        out_path=export_dir,
        out_name=f"{base}.csv"
    )

    print(f"Done: {os.path.basename(out_tbl)}  |  XLSX: {xlsx_path}  |  CSV: {csv_path}")

print("All zonal tables created with VALID_AREA_M2/KM2 based on 0.1° pixels and exported to Excel/CSV.")


In [ ]:
#POST-PROCESSING IN ARGISPRO-PYTHON API

In [ ]:
#Depending on the computional memory, the user can run them once as seen below or one by one as demostrated in the next two cells

In [ ]:
import arcpy, os, math
from arcpy.sa import ZonalStatisticsAsTable, Con, Raster

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

# --- Paths ---
gdb = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
arcpy.env.workspace = gdb

# Feature class with zones
latbands_fc = r"U:\\class_transition\landcover_transition\landcover_transition.gdb\Study_tse"

# Zone field
zone_field = "Name"

# Original input rasters (Fall example here)
input_rasters = {
    "wint": "wint_imb_wm2_clean",
    "wint": "wint_sens_wm2_clean",
    "wint": "wint_latent_wm2_clean"
}

# Env: use native raster cell size
arcpy.env.cellSize = 0.1

# --- Constants for pixel area calculation ---
R = 6371008.7714  # meters
deg2rad = math.pi / 180.0
dlat_deg = 0.1
dlon_deg = 0.1
dlon_rad = dlon_deg * deg2rad

def mid_lat_from_band(band_text):
    """ Parse '44–45N' -> 44.5. """
    s = band_text.replace("–", "-").replace("N", "").strip()
    a, b = s.split("-")
    return (float(a) + float(b)) / 2.0

def pixel_area_m2_at_lat(lat_deg):
    """ Exact spherical formula for a 0.1° x 0.1° cell at given latitude. """
    phi1 = (lat_deg - dlat_deg/2.0) * deg2rad
    phi2 = (lat_deg + dlat_deg/2.0) * deg2rad
    return (R**2) * dlon_rad * (math.sin(phi2) - math.sin(phi1))

# --- Precompute centroid latitudes for zones ---
ecozone_lat = {}
if zone_field != "Lat_band":
    with arcpy.da.SearchCursor(latbands_fc, [zone_field, "SHAPE@XY"]) as sc:
        for zid, (x, y) in sc:
            ecozone_lat[zid] = y  # centroid latitude

# Where to write Excel / CSV
export_dir = r"X:\\"

# --- Step 1: Generate cooling/warming rasters ---
derived_rasters = []

# Latent (special sign flip)
lat = Raster(os.path.join(gdb, input_rasters["latent"]))
latent_cooling = Con(lat > 0, -lat, 0)
latent_warming = Con(lat < 0, -lat, 0)
latent_cooling_name = "wint_latent_cooling"
latent_warming_name = "wint_latent_warming"
latent_cooling.save(os.path.join(gdb, latent_cooling_name))
latent_warming.save(os.path.join(gdb, latent_warming_name))
derived_rasters += [latent_cooling_name, latent_warming_name]

# Sensible
sens = Raster(os.path.join(gdb, input_rasters["sens"]))
sens_cooling = Con(sens < 0, sens, 0)
sens_warming = Con(sens > 0, sens, 0)
sens_cooling_name = "wint_sens_cooling"
sens_warming_name = "wint_sens_warming"
sens_cooling.save(os.path.join(gdb, sens_cooling_name))
sens_warming.save(os.path.join(gdb, sens_warming_name))
derived_rasters += [sens_cooling_name, sens_warming_name]

# Imbalance (Ground Heat Fluxes)
imb = Raster(os.path.join(gdb, input_rasters["imb"]))
imb_cooling = Con(imb < 0, imb, 0)
imb_warming = Con(imb > 0, imb, 0)
imb_cooling_name = "wint_imb_cooling"
imb_warming_name = "wint_imb_warming"
imb_cooling.save(os.path.join(gdb, imb_cooling_name))
imb_warming.save(os.path.join(gdb, imb_warming_name))
derived_rasters += [imb_cooling_name, imb_warming_name]

print(f"Derived rasters created: {derived_rasters}")


# --- Step 2: Loop over derived rasters for ZonalStats ---
for rname in derived_rasters:
    in_ras = os.path.join(gdb, rname)

    # Distinguish output table name
    if zone_field == "Lat_band":
        out_tbl = os.path.join(gdb, f"{rname}_zstats_latbands")
    else:
        out_tbl = os.path.join(gdb, f"eco_{rname}_zstats")

    # --- Run Zonal Statistics ---
    ZonalStatisticsAsTable(
        in_zone_data=latbands_fc,
        zone_field=zone_field,
        in_value_raster=in_ras,
        out_table=out_tbl,
        ignore_nodata="DATA",
        statistics_type="ALL"
    )

    # --- Add area fields ---
    add_fields = [
        ("LAT_MID", "DOUBLE"),
        ("PIX_AREA_M2", "DOUBLE"),
        ("VALID_AREA_M2", "DOUBLE"),
        ("VALID_AREA_KM2", "DOUBLE")
    ]
    existing = {f.name for f in arcpy.ListFields(out_tbl)}
    for fname, ftype in add_fields:
        if fname not in existing:
            arcpy.management.AddField(out_tbl, fname, ftype)

    # --- Update areas ---
    with arcpy.da.UpdateCursor(
        out_tbl, [zone_field, "COUNT", "LAT_MID", "PIX_AREA_M2", "VALID_AREA_M2", "VALID_AREA_KM2"]
    ) as cur:
        for zname, cnt, lat_mid, pix_m2, area_m2, area_km2 in cur:
            if zone_field == "Lat_band":
                lat_mid = mid_lat_from_band(zname)
            else:
                lat_mid = ecozone_lat.get(zname, 0.0)

            pix_m2  = pixel_area_m2_at_lat(lat_mid)
            area_m2 = (cnt or 0) * pix_m2
            area_km2 = area_m2 / 1e6
            cur.updateRow((zname, cnt, lat_mid, pix_m2, area_m2, area_km2))

    # --- Exports ---
    base = os.path.splitext(os.path.basename(out_tbl))[0]

    xlsx_path = os.path.join(export_dir, f"{base}.xlsx")
    arcpy.conversion.TableToExcel(
        Input_Table=out_tbl,
        Output_Excel_File=xlsx_path,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )

    csv_path = os.path.join(export_dir, f"{base}.csv")
    arcpy.conversion.TableToTable(
        in_rows=out_tbl,
        out_path=export_dir,
        out_name=f"{base}.csv"
    )

    print(f" Done: {os.path.basename(out_tbl)}  |  XLSX: {xlsx_path}  |  CSV: {csv_path}")

print("🎉 All zonal tables (cooling & warming separated) created and exported.")


In [ ]:
#The user can rerun them one by one if running all crash due to thier computation memory

In [ ]:
import arcpy, os, math
from arcpy.sa import ZonalStatisticsAsTable, Con, Raster

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

# --- Paths ---
gdb = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
arcpy.env.workspace = gdb

# Feature class with zones
latbands_fc = r"U:\\class_transition\landcover_transition\landcover_transition.gdb\Study_tse"

# Zone field
zone_field = "Name"

# Only Imbalance raster
imb_raster = "sum_imb_wm2_clean"

# Env: use native raster cell size
arcpy.env.cellSize = 0.1

# --- Constants for pixel area calculation ---
R = 6371008.7714  # meters
deg2rad = math.pi / 180.0
dlat_deg = 0.1
dlon_deg = 0.1
dlon_rad = dlon_deg * deg2rad

def mid_lat_from_band(band_text):
    """ Parse '44–45N' -> 44.5. """
    s = band_text.replace("–", "-").replace("N", "").strip()
    a, b = s.split("-")
    return (float(a) + float(b)) / 2.0

def pixel_area_m2_at_lat(lat_deg):
    """ Exact spherical formula for a 0.1° x 0.1° cell at given latitude. """
    phi1 = (lat_deg - dlat_deg/2.0) * deg2rad
    phi2 = (lat_deg + dlat_deg/2.0) * deg2rad
    return (R**2) * dlon_rad * (math.sin(phi2) - math.sin(phi1))

# --- Precompute centroid latitudes for zones ---
ecozone_lat = {}
if zone_field != "Lat_band":
    with arcpy.da.SearchCursor(latbands_fc, [zone_field, "SHAPE@XY"]) as sc:
        for zid, (x, y) in sc:
            ecozone_lat[zid] = y  # centroid latitude

# Where to write Excel / CSV
export_dir = r"X:\\"

# --- Step 1: Generate cooling/warming rasters for Imbalance ---
imb = Raster(os.path.join(gdb, imb_raster))
#imb_cooling = Con(imb < 0, imb, 0)
imb_warming = Con(imb > 0, imb, 0)

#imb_cooling_name = "sum_imb_cooling"
imb_warming_name = "sum_imb_warming"

#imb_cooling.save(imb_cooling_name)   # save directly into gdb
imb_warming.save(imb_warming_name)

#derived_rasters = [imb_cooling_name, imb_warming_name]
derived_rasters = [imb_warming_name]
print(f" Derived rasters created: {derived_rasters}")

# --- Step 2: Loop over derived rasters for ZonalStats ---
for rname in derived_rasters:
    in_ras = rname  # workspace already set to gdb

    # Distinguish output table name
    if zone_field == "Lat_band":
        out_tbl = f"{rname}_zstats_latbands"
    else:
        out_tbl = f"eco_{rname}_zstats"

    # --- Run Zonal Statistics ---
    ZonalStatisticsAsTable(
        in_zone_data=latbands_fc,
        zone_field=zone_field,
        in_value_raster=in_ras,
        out_table=out_tbl,
        ignore_nodata="DATA",
        statistics_type="ALL"
    )

    # --- Add area fields ---
    add_fields = [
        ("LAT_MID", "DOUBLE"),
        ("PIX_AREA_M2", "DOUBLE"),
        ("VALID_AREA_M2", "DOUBLE"),
        ("VALID_AREA_KM2", "DOUBLE")
    ]
    existing = {f.name for f in arcpy.ListFields(out_tbl)}
    for fname, ftype in add_fields:
        if fname not in existing:
            arcpy.management.AddField(out_tbl, fname, ftype)

    # --- Update areas ---
    with arcpy.da.UpdateCursor(
        out_tbl, [zone_field, "COUNT", "LAT_MID", "PIX_AREA_M2", "VALID_AREA_M2", "VALID_AREA_KM2"]
    ) as cur:
        for zname, cnt, lat_mid, pix_m2, area_m2, area_km2 in cur:
            if zone_field == "Lat_band":
                lat_mid = mid_lat_from_band(zname)
            else:
                lat_mid = ecozone_lat.get(zname, 0.0)

            pix_m2  = pixel_area_m2_at_lat(lat_mid)
            area_m2 = (cnt or 0) * pix_m2
            area_km2 = area_m2 / 1e6
            cur.updateRow((zname, cnt, lat_mid, pix_m2, area_m2, area_km2))

    # --- Exports ---
    base = os.path.splitext(os.path.basename(out_tbl))[0]

    xlsx_path = os.path.join(export_dir, f"{base}.xlsx")
    arcpy.conversion.TableToExcel(
        Input_Table=out_tbl,
        Output_Excel_File=xlsx_path,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )

    csv_path = os.path.join(export_dir, f"{base}.csv")
    arcpy.conversion.TableToTable(
        in_rows=out_tbl,
        out_path=export_dir,
        out_name=f"{base}.csv"
    )

    print(f"Done: {out_tbl}  |  XLSX: {xlsx_path}  |  CSV: {csv_path}")

print(" Imbalance warming zonal tables created and exported.")


In [ ]:
#from the export (net exchange trend grid, i.e, cooling and warming signed sumed)
#generated from above step(s), we then decompose them per ecozone. See decomposition steps below

In [ ]:
import arcpy, os, csv
from arcpy.sa import Raster, IsNull, Con, ZonalStatisticsAsTable
arcpy.CheckOutExtension("Spatial")

# ===================== EDIT THESE PATHS/NAMES =====================
GDB         = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
ZONE_RAS    = "Study_tse_Raster"      # integer zone raster with values 1..9
ZONE_FIELD  = "Value"                 # field name that stores zone IDs ('Value'/'VALUE')
NET_RASTER  = "Net_exchange2"         # + = warming, - = cooling, NoData = none significant
OUT_CSV     = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\net2_zone_summary.csv"
OUT_TOTALS  = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\net2_grand_totals.csv"

# Optional pretty names for your 1..9 codes:
eco_name = {1:"SA",2:"TP",3:"BS",4:"BC",5:"TC",6:"HP",7:"TSW",8:"TCO",9:"TSE"}

# ===================== ENVIRONMENT =====================
arcpy.env.workspace       = GDB
arcpy.env.overwriteOutput = True
# Align to zone raster so counts/areas line up exactly with ecozone footprint
arcpy.env.snapRaster      = ZONE_RAS
arcpy.env.extent          = arcpy.Describe(ZONE_RAS).extent
arcpy.env.cellSize        = arcpy.Describe(ZONE_RAS).meanCellWidth

# Helper to find a field name robustly (case differences, *_SUM variants, etc.)
def _find_field(tbl, target_upper, fallback_suffix=None):
    fields = [f.name for f in arcpy.ListFields(tbl)]
    for nm in fields:
        if nm.upper() == target_upper:
            return nm
    if fallback_suffix:
        for nm in fields:
            if nm.upper().endswith(fallback_suffix):
                return nm
    return None

# ===================== 1) TOTAL PIXELS PER ZONE (no RAT needed) =====================
# Build a ones raster wherever the zone raster has data; sum ones per zone -> total pixel counts
zone = Raster(ZONE_RAS)
ones = Con(IsNull(zone), 0, 1)

tot_tbl = os.path.join(GDB, "ZS_zone_totalpix")
if arcpy.Exists(tot_tbl):
    arcpy.Delete_management(tot_tbl)
ZonalStatisticsAsTable(ZONE_RAS, ZONE_FIELD, ones, tot_tbl, "DATA", "SUM")

sum_field = _find_field(tot_tbl, "SUM", fallback_suffix="_SUM")
if not sum_field:
    raise RuntimeError("Could not find a SUM field in total-pixel table.")

total_pixels = {}
with arcpy.da.SearchCursor(tot_tbl, [ZONE_FIELD, sum_field]) as cur:
    for zid, s in cur:
        total_pixels[int(zid)] = int(round(s or 0))

# ===================== 2) MASKS & VALUE RASTERS FROM NET =====================
net     = Raster(NET_RASTER)
valid01 = Con(IsNull(net), 0, 1)  # 1 where Net has data
warm01  = Con(net > 0, 1, 0)      # 1 where Net > 0 (warming)
cool01  = Con(net < 0, 1, 0)      # 1 where Net < 0 (cooling)

# For sums of values by sign
warm_vals = Con(net > 0, net, 0)  # keep positive values, 0 elsewhere
cool_vals = Con(net < 0, net, 0)  # keep negative values, 0 elsewhere

def zonal_sum_to_gdb(value_raster, name):
    """Return dict {zone_id: SUM(value_raster)} written via ZonalStatisticsAsTable."""
    tbl = os.path.join(GDB, name)
    if arcpy.Exists(tbl):
        arcpy.Delete_management(tbl)
    ZonalStatisticsAsTable(ZONE_RAS, ZONE_FIELD, value_raster, tbl, "DATA", "SUM")
    if not arcpy.Exists(tbl):
        raise RuntimeError(f"ZonalStatisticsAsTable failed for {name}:\n{arcpy.GetMessages(2)}")
    sfield = _find_field(tbl, "SUM", fallback_suffix="_SUM")
    if not sfield:
        raise RuntimeError(f"SUM field not found in {tbl}. Fields: {[f.name for f in arcpy.ListFields(tbl)]}")
    out = {}
    with arcpy.da.SearchCursor(tbl, [ZONE_FIELD, sfield]) as cur:
        for zid, s in cur:
            out[int(zid)] = float(s or 0.0)
    return out

def zonal_mean_to_gdb(value_raster, name):
    """Return dict {zone_id: MEAN(value_raster)} (NoData excluded)."""
    tbl = os.path.join(GDB, name)
    if arcpy.Exists(tbl):
        arcpy.Delete_management(tbl)
    ZonalStatisticsAsTable(ZONE_RAS, ZONE_FIELD, value_raster, tbl, "DATA", "MEAN")
    if not arcpy.Exists(tbl):
        raise RuntimeError(f"ZonalStatisticsAsTable failed for {name}:\n{arcpy.GetMessages(2)}")
    out = {}
    with arcpy.da.SearchCursor(tbl, [ZONE_FIELD, "MEAN"]) as cur:
        for zid, m in cur:
            out[int(zid)] = float(m)
    return out

# Pixel counts via SUM on 0/1 masks
valid_counts = {z:int(round(v)) for z,v in zonal_sum_to_gdb(valid01, "ZS_valid01").items()}
warm_counts  = {z:int(round(v)) for z,v in zonal_sum_to_gdb(warm01,  "ZS_warm01").items()}
cool_counts  = {z:int(round(v)) for z,v in zonal_sum_to_gdb(cool01,  "ZS_cool01").items()}

# Sums of raster values (positive-only and negative-only)
warm_sum = zonal_sum_to_gdb(warm_vals, "ZS_warm_sum")  # positive sums
cool_sum = zonal_sum_to_gdb(cool_vals, "ZS_cool_sum")  # negative sums

# Means of Net (over valid pixels only)
net_mean = zonal_mean_to_gdb(net, "ZS_net_mean")

# ===================== 3) WRITE PER-ZONE CSV =====================
with open(OUT_CSV, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow([
        "ZoneID","Ecozone","Total_Pixels",
        "Warm_Pixels","Warm_%","Cool_Pixels","Cool_%",
        "Valid_Pixels","Valid_%","NoData_Pixels","NoData_%",
        "Sum_Warming_Wm2yr","Sum_Cooling_Wm2yr","Net_Mean_Wm2yr"
    ])
    for zid in sorted(total_pixels.keys()):
        tot  = total_pixels.get(zid, 0)
        warm = warm_counts.get(zid, 0)
        cool = cool_counts.get(zid, 0)
        val  = valid_counts.get(zid, 0)
        nd   = max(tot - val, 0)

        if tot > 0:
            warm_pct  = 100.0 * warm / tot
            cool_pct  = 100.0 * cool / tot
            valid_pct = 100.0 * val  / tot
            nodata_pct= 100.0 - valid_pct
        else:
            warm_pct = cool_pct = valid_pct = nodata_pct = 0.0

        w.writerow([
            zid, eco_name.get(zid, str(zid)), tot,
            warm, round(warm_pct,3), cool, round(cool_pct,3),
            val, round(valid_pct,3), nd, round(nodata_pct,3),
            round(warm_sum.get(zid, 0.0), 6),
            round(cool_sum.get(zid, 0.0), 6),
            round(net_mean.get(zid, float("nan")), 6)
        ])

print("Per-zone summary written:", OUT_CSV)

# ===================== 4) WRITE GRAND TOTALS CSV =====================
grand_warm_sum = sum(warm_sum.values())
grand_cool_sum = sum(cool_sum.values())
with open(OUT_TOTALS, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["Metric","Value","Units"])
    w.writerow(["Total_warming_sum", round(grand_warm_sum, 6), "W m^-2 yr^-1 (sum of pixel values >0)"])
    w.writerow(["Total_cooling_sum", round(grand_cool_sum, 6), "W m^-2 yr^-1 (sum of pixel values <0)"])

print("Grand totals written:", OUT_TOTALS)

# ===================== (OPTIONAL) AREA-INTEGRATED TOTALS =====================


In [ ]:
#====TO GET AREA-INTEGRATED TOTALS WE IMPLEMENTED THE STEPS BELOW =====

In [ ]:
import arcpy, os, csv
from arcpy.sa import Raster, IsNull, Con, ZonalStatisticsAsTable
arcpy.CheckOutExtension("Spatial")

# --- EDIT THESE WHEN YOU WANT TO SPECIFY PATHS  ---
GDB         = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\ENERGY_FLUXES_NEWCOLLECTIONS.gdb"
ZONE_RAS    = "Study_tse_Raster"   # integer zone raster (1..9)
ZONE_FIELD  = "Value"              # 'Value' or 'VALUE'
NET_RASTER  = "Net_exchange2"
OUT_CSV     = r"X:\ENERGY_FLUXES_NEWCOLLECTIONS\net2_zone_percentages.csv"

eco_name = {1:"SA",2:"TP",3:"BS",4:"BC",5:"TC",6:"HP",7:"TSW",8:"TCO",9:"TSE"}

arcpy.env.workspace       = GDB
arcpy.env.overwriteOutput = True
arcpy.env.snapRaster      = ZONE_RAS
arcpy.env.extent          = arcpy.Describe(ZONE_RAS).extent
arcpy.env.cellSize        = arcpy.Describe(ZONE_RAS).meanCellWidth

# Ensure RAT exists & get total pixel counts per zone
arcpy.management.BuildRasterAttributeTable(ZONE_RAS, "OVERWRITE")
fields = [f.name for f in arcpy.ListFields(ZONE_RAS)]
val_f  = ZONE_FIELD if ZONE_FIELD in fields else next(f for f in fields if f.lower()=="value")
cnt_f  = "COUNT" if "COUNT" in fields else next(f for f in fields if f.lower()=="count")

total_pixels = {}
with arcpy.da.SearchCursor(ZONE_RAS, [val_f, cnt_f]) as cur:
    for zid, cnt in cur:
        total_pixels[int(zid)] = int(cnt or 0)

# Masks from Net_exchange2
net = Raster(NET_RASTER)
valid01 = Con(IsNull(net), 0, 1)
warm01  = Con(net > 0, 1, 0)
cool01  = Con(net < 0, 1, 0)

def zonal_sum_to_gdb(mask, name):
    """Write ZonalStatisticsAsTable output to the GDB (not in_memory) and read SUM."""
    tbl = os.path.join(GDB, name)
    if arcpy.Exists(tbl):
        arcpy.Delete_management(tbl)
    ZonalStatisticsAsTable(ZONE_RAS, val_f, mask, tbl, "DATA", "SUM")
    # sanity: if tool produced no table or no rows, raise with messages
    if not arcpy.Exists(tbl):
        raise RuntimeError(f"ZonalStatisticsAsTable failed for {name}:\n{arcpy.GetMessages(2)}")
    # find SUM field reliably
    sum_field = None
    for f in arcpy.ListFields(tbl):
        if f.name.upper() == "SUM":
            sum_field = f.name; break
    if sum_field is None:
        for f in arcpy.ListFields(tbl):
            if f.name.upper().endswith("_SUM") or f.name.upper()=="SUM_":
                sum_field = f.name; break
    if sum_field is None:
        raise RuntimeError(f"SUM field not found in {tbl}. Fields: {[f.name for f in arcpy.ListFields(tbl)]}")
    # read counts
    out = {}
    with arcpy.da.SearchCursor(tbl, [val_f, sum_field]) as cur:
        for zid, s in cur:
            out[int(zid)] = int(round(s or 0))
    return out

valid_counts = zonal_sum_to_gdb(valid01, "ZS_valid01")
warm_counts  = zonal_sum_to_gdb(warm01,  "ZS_warm01")
cool_counts  = zonal_sum_to_gdb(cool01,  "ZS_cool01")

# Zonal MEAN of Net (over DATA only), also to GDB
zs_mean_tbl = os.path.join(GDB, "ZS_net_mean")
if arcpy.Exists(zs_mean_tbl):
    arcpy.Delete_management(zs_mean_tbl)
ZonalStatisticsAsTable(ZONE_RAS, val_f, NET_RASTER, zs_mean_tbl, "DATA", "MEAN")

mean_by_zone = {}
with arcpy.da.SearchCursor(zs_mean_tbl, [val_f, "MEAN"]) as cur:
    for zid, m in cur:
        mean_by_zone[int(zid)] = float(m)

# Assemble & write CSV
with open(OUT_CSV, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["ZoneID","Ecozone","Total_Pixels",
                "Valid_Pixels","Valid_%","NoData_Pixels","NoData_%",
                "Warm_Pixels","Warm_%","Cool_Pixels","Cool_%",
                "Net_Mean_Wm2yr"])
    for zid in sorted(total_pixels.keys()):
        tot  = total_pixels.get(zid, 0)
        val  = valid_counts.get(zid, 0)
        nd   = max(tot - val, 0)
        warm = warm_counts.get(zid, 0)
        cool = cool_counts.get(zid, 0)
        if tot > 0:
            valid_pct = 100.0 * val  / tot
            nodata_pct= 100.0 * nd   / tot
            warm_pct  = 100.0 * warm / tot
            cool_pct  = 100.0 * cool / tot
        else:
            valid_pct = nodata_pct = warm_pct = cool_pct = 0.0
        w.writerow([zid, eco_name.get(zid, str(zid)), tot,
                    val, round(valid_pct,3), nd, round(nodata_pct,3),
                    warm, round(warm_pct,3), cool, round(cool_pct,3),
                    round(mean_by_zone.get(zid, float("nan")), 6)])
print("Done:", OUT_CSV)


In [ ]:
# Codes for Ecozone level correlation between NDVI trend and temperature trend 
#Codes for Ecozone level correlation between NDVI trend and Net-Annual energy trend fluxes, 
#From SECTION A2, we computed the correlation between NDVI trend and Net-Annual energy trend fluxes
#From SECTION c2, we computed the Global level comparison: Welch t-test (unequal variances) & Mann–Whitney U (non-parametric).


In [ ]:
#SECTION A2: Here we present the correlation code we used to globally correlate temperature trend versus the cooling and warming energy trend fluxes
#This particular task was implemented directly in the Google Earth Engine
#HERE we presnetd Pipeline for cooling energy flux trend  versus temperature
#BUT TO CORRELATE FOR WARMING TREND, SWITCHING COOLING_total to WARMING_total will get the results for temperature trend versus the warming energy trend fluxes
#See README file for information on these grids and proper download for verifications

// ===========================================================
// Mean T2m (°C), 1986–2023 — clipped to study area & aligned to Cooling grid
// ===========================================================

// ---- INPUTS ----
var ZONES_ASSET = 'projects/danielsiglab/assets/study_area_tse_tw';
var COOL_ASSET  = 'projects/danielsiglab/assets/Cooling_total'; #BUT TO CORRELATE FOR WARMING TREND, SWITCH COOLING_total to WARMING_total 
var START_DATE  = '1986-01-01';
var END_DATE    = '1990-12-31';

// ---- LOADS ----
var zones   = ee.FeatureCollection(ZONES_ASSET);
var region  = zones.geometry();                   // <-- no buffer(0)
var cooling = ee.Image(COOL_ASSET).select(0);

var coolProj = cooling.projection();
var SCALE    = coolProj.nominalScale();

// ERA5-Land monthly temperatures
var era5 = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY')
  .filterDate(START_DATE, END_DATE);

// ---- BUILD MULTI-YEAR MEAN TEMPERATURE (°C) ----
var Tmean_C = era5
  .select('temperature_2m')        // Kelvin
  .mean()                          // mean of all months 1986–2023 (NoData ignored)
  .subtract(273.15)                // -> °C
  .rename('Tmean_1986_2023_C')
  .clip(zones);                    // clip to study area polygons

// ---- PREVIEW ----
Map.centerObject(zones, 4);
Map.addLayer(
  Tmean_C,
  {min: -30, max: 30, palette: ['#313695','#74add1','#ffffbf','#f46d43','#a50026']},
  'Mean T (°C) 1986–2023 — clipped'
);

// ---- EXPORT TO ASSET (aligned to Cooling_total grid) ----
var OUT_ASSET_ID = 'projects/danielsiglab/assets/TmeanC_1986_2023_studyarea_coolingGrid';

Export.image.toAsset({
  image: Tmean_C,
  description: 'Export_TmeanC_1986_2023_studyarea_coolingGrid',
  assetId: OUT_ASSET_ID,
  region: region,      // clipped extent
  crs: coolProj,       // align projection to Cooling_total
  scale: SCALE,        // align resolution to Cooling_total
  maxPixels: 1e13
});


// ==== CORRELATION: Cooling_total (X) vs mean T (Y) per ecozone (Name) ====

// ==== CORRELATION: Cooling_total (X) vs mean T (Y) per ecozone (Name) ====

// ==== CORRELATION: Cooling_total (X) vs mean T (Y) per ecozone (Name) ====

// ==== CORRELATION: Cooling_total (X) vs mean T (Y) per ecozone (Name) ====

// 1) Align mean temperature to Cooling_total grid and scale
var Tmean_aligned = Tmean_C
  .rename('Y')
  .resample('bilinear')
  .reproject({crs: coolProj, scale: SCALE});  // exact pixel alignment

// 2) Pair the bands and co-mask
var pair = ee.Image.cat(
  cooling.rename('X'),
  Tmean_aligned
).updateMask(cooling.mask().and(Tmean_aligned.mask()));

// helper: safely pull rho from any reducer key
function safeGetRho(dict) {
  dict = ee.Dictionary(dict);
  var keys = ee.List(dict.keys());
  // Prefer 'spearmansCorrelation', else 'correlation', else first value if any
  return ee.Algorithms.If(
    keys.contains('spearmansCorrelation'), dict.get('spearmansCorrelation'),
    ee.Algorithms.If(
      keys.contains('correlation'), dict.get('correlation'),
      ee.Algorithms.If(keys.size().gt(0), dict.values().get(0), null)
    )
  );
}

// 3) Spearman ρ per polygon
var perZone = zones.map(function (f) {
  // buffer by half a pixel so boundary pixels are counted
  var geom = f.geometry().buffer(ee.Number(SCALE).divide(2));

  // robust pair count: sum of ones over the pair mask
  var n = ee.Number(
    ee.Image.constant(1).rename('ones')
      .updateMask(pair.select('X').mask())           // co-mask already enforced
      .reduceRegion({
        reducer: ee.Reducer.sum(),
        geometry: geom,
        scale: SCALE,
        maxPixels: 1e13,
        bestEffort: true
      })
      .get('ones')
  );

  // compute spearman only if n >= 2
  var rho = ee.Algorithms.If(
    n.gte(2),
    safeGetRho(
      pair.reduceRegion({
        reducer: ee.Reducer.spearmansCorrelation(),
        geometry: geom,
        scale: SCALE,
        maxPixels: 1e13,
        bestEffort: true
      })
    ),
    null
  );

  return ee.Feature(null, {
    zone_name: ee.String(f.get('Name')),
    n: n,
    rho: rho,
    note: ee.String(ee.Algorithms.If(n.gte(2), 'ok', 'insufficient_n'))
  });
});

// 4) Preview the actual table (be sure you're printing perZone, not zones)
print('Spearman ρ per ecozone (Cooling_total vs mean T 1986–2023):', perZone.limit(9));
print('First row values:', ee.Feature(perZone.first()).toDictionary(['zone_name','n','rho','note']));


// 5) EXPORT table to Drive (as before)
Export.table.toDrive({
  collection: perZone,
  description: 'Cooling_vs_Tmean_1986_2023_Spearman_perZoneName',
  folder: 'Cooling_Total_Correlation',
  fileFormat: 'CSV'
});



In [ ]:

#SECTION B2: Codes for Ecozone level correlation between NDVI trend and Net-Annual energy trend fluxes
#we implemented this in ARGISPro python-API extension



import arcpy
from arcpy.sa import ExtractMultiValuesToPoints
import numpy as np
import pandas as pd
from scipy.stats import kendalltau   # Kendall τ

arcpy.env.overwriteOutput = True

# ------------------------------------------------------------------
# 0. PATHS – Specicy path of your Geodatabase
# ------------------------------------------------------------------
gdb = r"U:\slope_sumarry\slope_sumarry.gdb" #path to the ARGISPro geodatabase(gdb)
arcpy.env.workspace = gdb

# polygon ecozone / study feature class:
ecozone_fc = "Study_tse"          # this is stduy are shapefile

# rasters (outside the gdb, in the same folder as before)
ndvi_ras = r"X:\Aggregated_ndvitrend.tif"  #This is the NDVI trend grid
net_ras  = r"X:\Annual_net_FLUX_TREND.tif" ##This is the net trend energy flux trend grid

# field in Study_tse we will use as ecozone label:
ecozone_name_field = "Name"     #The is the shapefile of ecozones

# how many random points to generate
number_of_points = 54000 #here the number can be varied for each separate runs

# ------------------------------------------------------------------
# 1. CHECK THAT INPUTS EXIST
# ------------------------------------------------------------------
print("Workspace:", arcpy.env.workspace)
print("Feature classes in GDB:")
for fc in arcpy.ListFeatureClasses():
    print("  -", fc)

if not arcpy.Exists(ecozone_fc):
    raise RuntimeError(f"Feature class '{ecozone_fc}' not found in {gdb}")

if not arcpy.Exists(ndvi_ras):
    raise RuntimeError(f"NDVI raster not found: {ndvi_ras}")

if not arcpy.Exists(net_ras):
    raise RuntimeError(f"NET raster not found:  {net_ras}")

# ------------------------------------------------------------------
# 2. CREATE RANDOM POINTS INSIDE Study_tse
# ------------------------------------------------------------------
sample_points_fc = gdb + r"\eco_rand_pts"

print("\nCreating random points…")

# IMPORTANT FIX: create_multipoint_output must be POINT or MULTIPOINT
arcpy.management.CreateRandomPoints(
    out_path=gdb,
    out_name="eco_rand_pts",
    constraining_feature_class=ecozone_fc,
    constraining_extent="",               # use polygon extent
    number_of_points_or_field=number_of_points,
    minimum_allowed_distance="0 Meters",
    create_multipoint_output="POINT",    # <-- FIXED
    multipoint_size=""
)

print("Random points created:", sample_points_fc)

# ------------------------------------------------------------------
# 3. SPATIAL JOIN: ADD ECOZONE Name TO EACH POINT
# ------------------------------------------------------------------
joined_points_fc = gdb + r"\eco_rand_pts_join"

print("\nRunning spatial join to attach ecozone Name…")

arcpy.analysis.SpatialJoin(
    target_features=sample_points_fc,
    join_features=ecozone_fc,
    out_feature_class=joined_points_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT"
)

print("Spatial join done:", joined_points_fc)

# ------------------------------------------------------------------
# 4. EXTRACT NDVI & NET RASTER VALUES TO THE POINTS
# ------------------------------------------------------------------
print("\nChecking out Spatial Analyst and extracting raster values…")
arcpy.CheckOutExtension("Spatial")

ExtractMultiValuesToPoints(
    in_point_features=joined_points_fc,
    in_rasters=[[ndvi_ras, "NDVI_trend"],
                [net_ras,  "NET_trend"]],
    bilinear_interpolate_values="NONE"
)

print("Raster values extracted into fields NDVI_trend and NET_trend.")

# ------------------------------------------------------------------
# 5. CONVERT TO PANDAS DATAFRAME
# ------------------------------------------------------------------
fields = [ecozone_name_field, "NDVI_trend", "NET_trend"]
records = []

with arcpy.da.SearchCursor(joined_points_fc, fields) as cursor:
    for row in cursor:
        records.append(row)

df = pd.DataFrame(records, columns=["eco_name", "NDVI", "NET"])

# Handle NoData (if -9999 is used) and drop NaNs
df = df.replace({-9999: np.nan})
df = df.dropna(subset=["NDVI", "NET"])

print("\nTotal valid points after removing NoData:", len(df))

# ------------------------------------------------------------------
# 6. KENDALL TAU PER ECOZONE WITH NDVI THRESHOLDS
# ------------------------------------------------------------------
ndvi_thresholds = [0.0, 0.0015, 0.004, 0.005, 0.006] #Here threholds of the ndvi trend can be specified

results = []

for eco, sub in df.groupby("eco_name"):
    for thr in ndvi_thresholds:
        # keep points with NDVI trend >= threshold
        sub_thr = sub[sub["NDVI"] >= thr]
        n = len(sub_thr)
        if n < 30:
            continue  # skip tiny samples

        tau, p = kendalltau(sub_thr["NDVI"], sub_thr["NET"])
        if np.isnan(tau):
            continue

        results.append({
            "eco_name": eco,
            "NDVI_min": thr,
            "N_points": n,
            "tau": float(tau),
            "p_value": float(p)
        })

res_df = pd.DataFrame(results)
res_df = res_df.sort_values(by=["eco_name", "NDVI_min"])

print("\nKendall tau per ecozone and NDVI-threshold:")
print(res_df.to_string(index=False))

# ------------------------------------------------------------------
# 7. OPTIONAL: EXPORT RESULTS TO CSV
# ------------------------------------------------------------------
out_csv = r"U:\slope_summary15\slope_summary\NDVI_NET_Kendall_per_Study_tse.csv"
res_df.to_csv(out_csv, index=False)
print("\nResults written to:", out_csv)





#SECTION C2: Codes for Global level comparison: Welch t-test (unequal variances) & Mann–Whitney U (non-parametric)
# Here we tested Is the NDVI–energy trend relationship and the energy trend itself different 
#between stable vegetation vs greening-transition pixels


import arcpy
from arcpy.sa import ExtractMultiValuesToPoints
import numpy as np
import pandas as pd
from scipy.stats import kendalltau, ttest_ind, mannwhitneyu
import datetime
import os

arcpy.env.overwriteOutput = False  # To avoid overwrite existing outputs

# ----------------------------------------------------------
# 0. PATHS / INPUTS
# ----------------------------------------------------------
# Geodatabase with COMBINEDVCGL (transition / stable codes)
gdb_transition = r"U:\class_transition\landcover_transition\landcover_transition.gdb"

# Geodatabase with NDVI trend (LINEARSLOPE_MOSAIC) and where we write points
gdb_slope = r"U:\slope_summary15\slope_sumarry\slope_sumarry.gdb"

# 30 m NDVI trend raster (in slope GDB)
ndvi_ras_path = os.path.join(gdb_slope, "LINEARSLOPE_MOSAIC")

# 30 m NET energy trend raster (resampled) – in separate GDB
net_ras_path = r"X:\ndvi_trend_enegry_correlations\ndvi_trend_enegry_correlations.gdb\Annual_net_FLUX_TRE_Resample12"

# Where to store CSV outputs
out_folder = r"U:\class_transition\landcover_transition"

# How many UNIQUE 30 m cells to sample (upper bound, no double sampling)
total_unique_cells_to_sample = 000  # adjust if you want more/less

# STABLE CODES (diagonals, including 505)
stable_codes = [101, 202, 303, 404, 505, 606, 707, 808, 909, 1010, 1111, 1212] #These codes including the greening codes are to be drawn from the transition grid "COMBINEDVCGL" which store categorical values of vegetation type transitions

# GREENING TRANSITION CODES (exact list you gave)
greening_codes = [
    106, 107, 108, 109, 110, 111, 112,
    206, 207, 208, 209, 210, 211, 212,
    306, 307, 308, 309, 310, 311, 312,
    405, 406, 407, 408, 409, 410, 411, 412,
    506, 507, 508, 509, 510, 511, 512,
    610, 611, 612,
    910, 911, 912
]

# Optional max per group after filtering (set to None to keep all)
max_per_group = 10000  # e.g. 20000 stable + 20000 greening; set to None to keep all. For each run user can vary it carefully based on available memory.

# Timestamp for unique names
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# ----------------------------------------------------------
# 1. FIND COMBINEDVCGL INSIDE THE TRANSITION GDB
# ----------------------------------------------------------
print("Checking COMBINEDVCGL in transition geodatabase...")
arcpy.env.workspace = gdb_transition

rasters = arcpy.ListRasters()
print("Rasters found in", gdb_transition, ":")
if rasters:
    for r in rasters:
        print("  -", r)
else:
    print("  (no rasters listed)")

if "COMBINEDVCGL" not in rasters:
    raise RuntimeError("COMBINEDVCGL not found in landcover_transition.gdb. "
                       "Check the exact raster name in ArcGIS Pro Contents.")

combined_ras_name = "COMBINEDVCGL" #this grid contains our landcover type transition values
combined_ras_path = os.path.join(gdb_transition, combined_ras_name)
combined_ras = arcpy.Raster(combined_ras_path)

# ----------------------------------------------------------
# 2. CHECK NDVI & NET INPUTS
# ----------------------------------------------------------
print("\nChecking NDVI and NET rasters...")

if not arcpy.Exists(ndvi_ras_path):
    raise RuntimeError(f"NDVI trend (LINEARSLOPE_MOSAIC) not found: {ndvi_ras_path}")
if not arcpy.Exists(net_ras_path):
    raise RuntimeError(f"NET trend (Annual_net_FLUX_TREND) not found: {net_ras_path}")

ndvi_ras = arcpy.Raster(ndvi_ras_path)
net_ras  = arcpy.Raster(net_ras_path)

# Make slope GDB the workspace for point outputs
arcpy.env.workspace = gdb_slope

# Sanity print cell sizes
print("COMBINEDVCGL cell size:", combined_ras.meanCellWidth, combined_ras.meanCellHeight)
print("NDVI cell size:", ndvi_ras.meanCellWidth, ndvi_ras.meanCellHeight)
print("NET cell size:", net_ras.meanCellWidth, net_ras.meanCellHeight)

arcpy.CheckOutExtension("Spatial")

# ----------------------------------------------------------
# 3. UNIQUE CELL SAMPLING (NO DOUBLE SAMPLING)
# ----------------------------------------------------------
print("\nSampling unique 30 m cells from COMBINEDVCGL grid...")

width  = combined_ras.width   # number of columns
height = combined_ras.height  # number of rows
n_total_cells = width * height
print(f"Raster grid: width={width}, height={height}, total cells={n_total_cells}")

n_sample = min(total_unique_cells_to_sample, n_total_cells)
print(f"Sampling {n_sample} UNIQUE cells (no replacement).")

rng = np.random.default_rng()
lin_idx = rng.choice(n_total_cells, size=n_sample, replace=False)

rows = lin_idx // width
cols = lin_idx % width
sampled_rc = np.column_stack((rows, cols))  # (n_sample, 2) [row, col]

# Convert row/col -> XY (cell centers)
desc = arcpy.Describe(combined_ras)
ext = desc.extent
cellsize_x = combined_ras.meanCellWidth
cellsize_y = combined_ras.meanCellHeight

x_min = ext.XMin
y_max = ext.YMax

# Create point FC for these unique sampled cells (in slope GDB)
pts_name = f"VCGL_uniqueCells_pts_{ts}"
pts_fc = os.path.join(gdb_slope, pts_name)

print("Creating point feature class:", pts_fc)

spatial_ref = combined_ras.spatialReference
arcpy.management.CreateFeatureclass(
    out_path=gdb_slope,
    out_name=pts_name,
    geometry_type="POINT",
    spatial_reference=spatial_ref
)

# Insert points (geometry only, no cell_id)
with arcpy.da.InsertCursor(pts_fc, ["SHAPE@XY"]) as icur:
    for (row, col) in sampled_rc:
        x = x_min + (col + 0.5) * cellsize_x
        y = y_max - (row + 0.5) * cellsize_y
        icur.insertRow([(x, y)])

print("Unique-cell points created:", pts_fc)

# ----------------------------------------------------------
# 4. EXTRACT COMBINEDVCGL, NDVI, NET TO THESE POINTS
# ----------------------------------------------------------
print("\nExtracting COMBINEDVCGL, NDVI_trend, NET_trend to unique-cell points...")

ExtractMultiValuesToPoints(
    in_point_features=pts_fc,
    in_rasters=[
        [combined_ras_path, "VCGL_val"],
        [ndvi_ras_path,     "NDVI_trend"],
        [net_ras_path,      "NET_trend"]
    ],
    bilinear_interpolate_values="NONE"
)

print("Fields VCGL_val, NDVI_trend, NET_trend added to:", pts_fc)

# ----------------------------------------------------------
# 5. BUILD DATAFRAME AND FILTER
# ----------------------------------------------------------
print("\nBuilding DataFrame and filtering valid samples...")

fields = ["VCGL_val", "NDVI_trend", "NET_trend", "SHAPE@XY"]
records = []
with arcpy.da.SearchCursor(pts_fc, fields) as cur:
    for vcgl, ndvi_val, net_val, (x, y) in cur:
        records.append((vcgl, ndvi_val, net_val, x, y))

df = pd.DataFrame(records, columns=["VCGL_val", "NDVI_trend", "NET_trend", "X", "Y"])

# Clean NoData if present (e.g. -9999) and drop NaNs
df = df.replace({-9999: np.nan})
df = df.dropna(subset=["VCGL_val", "NDVI_trend", "NET_trend"])

# Remove background / non-vegetation if coded as 0
df = df[df["VCGL_val"] != 0]

print("Total valid unique-cell samples after filtering:", len(df))

# ----------------------------------------------------------
# 6. SPLIT INTO STABLE vs GREENING
# ----------------------------------------------------------
df_stable = df[df["VCGL_val"].isin(stable_codes)].copy()
df_green  = df[df["VCGL_val"].isin(greening_codes)].copy()

print("  STABLE samples (before downsampling):   ", len(df_stable))
print("  GREENING samples (before downsampling): ", len(df_green))

if len(df_stable) < 10:
    raise RuntimeError("Too few STABLE samples after filtering; increase total_unique_cells_to_sample.")
if len(df_green) < 10:
    raise RuntimeError("Too few GREENING samples after filtering; increase total_unique_cells_to_sample.")

# Optional: downsample to limit per group
def downsample_group(df_group, n_max):
    if (n_max is None) or (len(df_group) <= n_max):
        return df_group
    return df_group.sample(n=n_max, random_state=42)

df_stable = downsample_group(df_stable, max_per_group)
df_green  = downsample_group(df_green, max_per_group)

print("After optional downsampling:")
print("  STABLE samples:   ", len(df_stable))
print("  GREENING samples: ", len(df_green))

# ----------------------------------------------------------
# 7. KENDALL τ: NDVI vs NET FOR EACH GROUP
# ----------------------------------------------------------
print("\nComputing Kendall τ between NDVI_trend and NET_trend...")

tau_stable, p_stable = kendalltau(df_stable["NDVI_trend"], df_stable["NET_trend"])
tau_green,  p_green  = kendalltau(df_green["NDVI_trend"],  df_green["NET_trend"])

print("\nKendall τ results (NDVI_trend vs NET_trend):")
print(f"  STABLE group:   tau = {tau_stable:.4f}, p = {p_stable:.3e}, N = {len(df_stable)}")
print(f"  GREENING group: tau = {tau_green:.4f}, p = {p_green:.3e}, N = {len(df_green)}")

# ----------------------------------------------------------
# 8. COMPARE NET_TREND BETWEEN GROUPS (t-test & Mann–Whitney)
# ----------------------------------------------------------
print("\nComparing NET_trend distributions (GREENING vs STABLE)...")

net_stable = df_stable["NET_trend"].values
net_green  = df_green["NET_trend"].values

# Welch t-test (unequal variances)
t_stat, p_t = ttest_ind(net_green, net_stable, equal_var=False, nan_policy="omit")

# Mann–Whitney U (non-parametric)
u_stat, p_u = mannwhitneyu(net_green, net_stable, alternative="two-sided")

print("Two-sample t-test (NET_trend, GREENING vs STABLE):")
print(f"  t = {t_stat:.4f}, p = {p_t:.3e}")

print("Mann–Whitney U test (NET_trend, GREENING vs STABLE):")
print(f"  U = {u_stat:.4f}, p = {p_u:.3e}")

# ----------------------------------------------------------
# 9. EXPORT CSVs
# ----------------------------------------------------------
print("\nExporting CSVs...")

csv_stable = os.path.join(out_folder, f"StableVCGL_NDVI_NET_uniqueCells_{ts}.csv")
csv_green  = os.path.join(out_folder, f"GreeningVCGL_NDVI_NET_uniqueCells_{ts}.csv")
csv_both   = os.path.join(out_folder, f"Stable_vs_Greening_NDVI_NET_uniqueCells_{ts}.csv")

df_stable.to_csv(csv_stable, index=False)
df_green.to_csv(csv_green, index=False)

df_all = pd.concat([
    df_stable.assign(group="stable"),
    df_green.assign(group="greening")
], ignore_index=True)
df_all.to_csv(csv_both, index=False)

print("CSV outputs:")
print("  STABLE:   ", csv_stable)
print("  GREENING: ", csv_green)
print("  BOTH:     ", csv_both)

print("\nDone.")






